### Calling the necessary packages

In [ ]:
using CoolProp
using CSV
using DataFrames
using SimplePCHIP
using Dates
using SpecialFunctions
using Plots
backend(:plotly)

### Function to solve for the temperature and flow field inside the HX

In [ ]:
function HX(Dₜ, PᵢoDₜ, PⱼoDₜ, NbL, NbR, Nₜₜ, Lₖ, tₛ, Bᵪ, tₖ, BₐL, BₐR, NF, Np, ṁₕ, T₂₍ᵢₙ₎, T₁₍ᵢₙ₎, Ratio, CaseRun, counter, Df, δfb, δft, Pf, Ct, Ncruc, ExD, P₁₍ᵢₙ₎, P₂₍ᵢₙ₎, ṁₗ, ttoDt, tsoDs, Hₜ, δsb, δtb, Nss, tsb, Bypass, FinType, NAF)

    Nₖ = 100 # Grid division in z-direction [-]
    Nⱼ = 100 # Grid division in y-direction [-]

    #Call initial solid properties
    Ts = [25.0,100.0,200.0,300.0,400.0,500.0,600.0,700.0,800.0,900.0,1000.0] # Temperature of Haynes282
    Cₚ₍ₛ₎i = [436.0,463.0,494.0,522.0,544.0,563.0,581.0,594.0,650.0,668.0,676.0] # Specific heat of Haynes282
    κₛi = [10.3,12.0,14.1,16.3,18.5,20.5,22.6,24.8,26.1,27.3,28.9] # Thermal conductivity of Haynes282
    αₛi = [0.00000288,0.00000315,0.00000348,0.00000381,0.00000413,0.00000444,0.00000473,0.00000509,0.00000488,0.00000498,0.00000521] # Thermal diffusivity of Haynes282
    ρₛi = zeros(11) # Density of Haynes282
    
    for a in 1:11
        ρₛi[a] = κₛi[a]/(αₛi[a]*Cₚ₍ₛ₎i[a])
    end
    
    Cpsitp = interpolate(Ts, Cₚ₍ₛ₎i)
    κsitp = interpolate(Ts,κₛi)
    αsitp = interpolate(Ts,αₛi)
    ρsitp = interpolate(Ts,ρₛi)
    
    
    #Parameter inputs
    Dₜ = Dₜ  # Tube outer diameter [m]
    tₜ = ttoDt*Dₜ  # Tube wall thickness [m]
    Dᵢₚ = Dₜ - (2*tₜ)  # Tube inner diameter [m]
    Dint = (Dₜ+Dᵢₚ)/2 # Intermediate tube diameter [m]
    PᵢoDₜ = PᵢoDₜ  # Pitch/diameter of tube in the x direction (columns) [-]
    PⱼoDₜ = PⱼoDₜ  # Pitch/diameter of tube in the y direction (rows) [-]
    Pᵢ = PᵢoDₜ * Dₜ  # Tube pitch in x direction [m]
    Pⱼ = PⱼoDₜ * Dₜ  # Tube pitch in y direction [m]
    NbL = NbL # Number of baffles in the left side of the HX [-]
    NbR = NbR # Number of baffles in the right side of the HX [-]
    Nₜₜ = Nₜₜ # Total number of tubes on one side of the HX [-]
    tₛ = tₛ # Thickness of splitter on one side [m]
    Lₖ = Lₖ  # Height of heat exchanger [m]
    Bᵪ = Bᵪ # Percentage of shell height occupied by tubes
    Df = Df # Fin diameter [m]
    δfb = δfb # Fin base thickness [m]
    δft = δfb # Fin tip thickness (same as fin base thickness for disc fins) [m]
    Pf = Pf # Fin pitch [m]
    Ct = Ct # Cruciform thickness
    Ch = (Dᵢₚ-Ct)/2 # Cruciform height
    
    if Ct<0.00000001
        Ct = 0
        Ncruc = 0
        Ch = 0
    end
    
    testPf = 0
    if Pf<0.0000001 || (Df/Dₜ)<1.0001
        testPf = 1
        Df = Dₜ
    end
    
    rₒₜ = Dₜ/2
    rf = Df/2
    rfc = rf + (δfb/2)
    
    DstempA = (2*Nₜₜ) + 1 + (2*tₛ/Pᵢ) - (4*Dₜ/Pⱼ) - (8*tₛ*Dₜ/(Pᵢ*Pⱼ))
    DstempB = ((2*Bᵪ/(Pᵢ*Pⱼ))*sqrt((1-(Bᵪ*Bᵪ))))
    DstempC = (sqrt((1-(Bᵪ*Bᵪ)))/Pᵢ) - (2*Bᵪ/Pⱼ) - ((4*Dₜ)*sqrt((1-(Bᵪ*Bᵪ)))/(Pᵢ*Pⱼ)) - (4*Bᵪ*tₛ/(Pᵢ*Pⱼ))
    Dₛ⁺ = ( (-DstempC + (sqrt((DstempC*DstempC) + (4*DstempA*DstempB)))) / (2*DstempB) )
    Dₛ⁻ = ( (-DstempC - (sqrt((DstempC*DstempC) + (4*DstempA*DstempB)))) / (2*DstempB) )
    
    Dₛ = min(Dₛ⁺,Dₛ⁻) # Shell inner diameter [m]
    
    if Dₛ<0
        Dₛ = max(Dₛ⁺,Dₛ⁻)
    end
    
    Nₜⱼ = round(Int64,(((4/Pⱼ)*((Bᵪ*Dₛ/2) - Dₜ)) + 1)) # Number of rows of tubes [-]
    Nₜᵢ = round(Int64,(((2/Pᵢ)*((Dₛ*sqrt((1-(Bᵪ*Bᵪ)))/2) - tₛ - (Pᵢ))) + 1))  # Number of columns of tubes [-]
    Lⱼ =  (((Nₜⱼ-1) * (Pⱼ/2))+ (2*Dₜ))  # Length of air flow [m]
    Lᵢ =  (((Nₜᵢ-1) * (Pᵢ/2))+(Pᵢ))  # Width [m]
    
    ExD = ExD # Additional space after inclusion of side longitudinal baffles
    NDₛ = Dₛ + ExD # New shell outer diameter after inclusion of side longitudinal baffles
    
    tₛₕ = tsoDs*NDₛ # Shell thickness [m]
    Dₒₛ = NDₛ + (2*tₛₕ) # Shell outer diameter [m]
    Pₓ = (sqrt((Pᵢ^2)+(Pⱼ^2)))/2 # Diagonal pitch [m]
    Lₕ = ((sqrt(((Dₛ/2)^2)-((Bᵪ*Dₛ/2)^2)))) # Hexagonal cross length on one side [m]
    Hₜ = Hₜ # Header thickness [m]
    NbT = NbL + NbR # Total number of baffles [-]
    
    if NF == 0
        NbT = NbL+NbR-1
    end
    
    Sₖ = zeros(NbT+2)  # Baffle spacing [m]
    
    geomviolate = 0
    
    #Check if parameters violate geometric constraints
    
    gap1 = Pₓ - Dₜ
    if gap1<0
        geomviolate = 1
    end
    
    gap2 = Pᵢ - Dₜ
    if gap2<0
        geomviolate = 1
    end
    
    gap3 = Pⱼ - Dₜ
    if gap3<0
        geomviolate = 1
    end
    
    gap4 = Pᵢ - Df
    if gap4<0
        geomviolate = 1
    end
    
    gap5 = Pⱼ - Df
    if gap5<0
        geomviolate = 1
    end
    
    gap6 = Pₓ - Df
    if gap6<0
        geomviolate = 1
    end
    
    if (Pᵢ/Dₜ)<=1.01
        geomviolate = 1
    end
    
    if (Pᵢ/Df)<=1.01
        geomviolate = 1
    end
    
    if (Pⱼ/Dₜ)<=1.01
        geomviolate = 1
    end
    
    if (Pⱼ/Df)<=1.01
        geomviolate = 1
    end
    
    if (Pₓ/Dₜ)<=1.01
        geomviolate = 1
    end
    
    if (Pₓ/Df)<=1.01
        geomviolate = 1
    end
    
    if Pf<δfb
        geomviolate = 1
    end
    
    if geomviolate == 1
        @goto endit
    end
    
    #Define baffle spacing in the HX
    LₖL = Lₖ - (NbL*tₖ)  # Length of the left side HX excluding baffle thickness [m]
    LₖR = Lₖ - (NbR*tₖ)  # Length of the left side HX excluding baffle thickness [m]
    
    LₖRR = Lₖ
    if NF == 0
        LₖRR = 0
        NbT = NbL+NbR-1
        Sₖ[1] = (2-BₐL)*Lₖ/(NbL+1)
        Sₖ[NbL+1] = (BₐL)*Lₖ/(NbL+1)
        dBL = (Sₖ[NbL+1]-Sₖ[1])/NbL # the common difference between baffles on the left side of the HX [m]
        
        for dtemp in 2:NbL
            Sₖ[dtemp] = Sₖ[1] + ((dtemp-1)*dBL)
        end
    else
        Sₖ[1] = (2-BₐL)*Lₖ/(NbL+1)
        Sₖ[NbL+1] = (BₐL)*Lₖ/(NbL+1)
        Sₖ[NbL+2] = (BₐR)*LₖRR/(NbR+1)
        Sₖ[NbT+2] = (2-BₐR)*LₖRR/(NbR+1)
        
        dBL = (Sₖ[NbL+1]-Sₖ[1])/NbL # the common difference between baffles on the left side of the HX [m]
        dBR = (Sₖ[NbT+2]-Sₖ[NbL+2])/NbR # the common difference between baffles on the right side of the HX [m]
        
        for dtemp in 2:NbL
            Sₖ[dtemp] = Sₖ[1] + ((dtemp-1)*dBL)
        end
        
        for dtemp in NbL+3:NbT+1
            Sₖ[dtemp] = Sₖ[NbL+2] + ((dtemp-NbL-2)*dBR)
        end
    end

    #Initialize thermophysical properties
    h₁ᵢ = PropsSI("H","T",(T₁₍ᵢₙ₎+273),"P",P₁₍ᵢₙ₎,"CO2") # Initial Enthalpy of sCO₂ in the shell [J/kg]
    h₂ᵢ = PropsSI("H","T",(T₂₍ᵢₙ₎+273),"P",P₁₍ᵢₙ₎,"CO2") # Initial Enthalpy of sCO₂ in the shell [J/kg]
    h1inew = PropsSI("H","T",(T₁₍ᵢₙ₎+273),"P",P₂₍ᵢₙ₎,"CO2")
    h2inew = PropsSI("H","T",(T₂₍ᵢₙ₎+273),"P",P₂₍ᵢₙ₎,"CO2")
    h1itemp = h₁ᵢ
    h2itemp = h₂ᵢ
    
    Q1 = abs((h1itemp-h2itemp)*ṁₕ)
    Q2 = abs((h1inew-h2inew)*ṁₗ)
    
    Qmax = min(Q1,Q2)
    
    P₁ = ones(Int(NbT+2))
    P₂ = ones(Int(NbT+2))
    
    a = 1
    for a in 1:(NbT+2)
        P₁[a] = P₁₍ᵢₙ₎
        P₂[a] = P₂₍ᵢₙ₎
    end
    
    #Variables used to define the flow correction factors
#     NF = NF # Number of folds [-]
#     Np = Np # Number of tube passes [-]
    Ncw = 0 # Number of tubes in the window [-]
    
    δsb = δsb # shell-to-baffle diametral clearance [-]
    lc = ((1-Bᵪ)/2)*Dₛ + (ExD/2) # Distance between the top of the baffle and the shell [m]
    θb = 2*acosd(1-(2*lc/NDₛ)) # angle between two radii intersected at the inside shell wall with the baffle cut [°]
    Asb = (π*(NDₛ-(2*tₛ))*δsb*(1-(θb/360)))/4 # shell-to-baffle leakage flow area [m²]
    
    Dctl = Dₛ-(3*Dₜ) # diameter of the circle through the centers of the outermost tubes [m]
    θctl = 2*acosd((Dₛ-(2*(lc-(ExD/2))))/Dctl) # angle between two radii intersected at the outer tube center with the baffle cut [°]
    δtb = δtb # tube-to-baffle diametral clearance [-]
    # Fw = (θctl/360) - (sind(θctl)/(2*π)) # Fw = 0 when no tubes are in the window [-]
    Fw = 0
    Atb = (π*Dₜ*δtb*Nₜₜ*(1-Fw))/2 # tube-to-baffle leakage flow area [m²]


    #Porosity, Specific surface are and hydraulic diameter
    XM₁ = ones(Int(Nⱼ),Int(Nₖ)) # Porosity of external sCO₂ [-]
    XM₂ = ones(Int(Nⱼ),Int(Nₖ)) # Porosity of internal sCO₂ [-]
    SW₁ = ones(Int(Nⱼ),Int(Nₖ)) # Specific surface area wetted by external sCO₂ [1/m]
    SW₂ = ones(Int(Nⱼ),Int(Nₖ)) # Specific surface area wetted by internal sCO₂ [1/m]
    DH₁ = ones(Int(Nⱼ),Int(Nₖ)) # Hydraulic diameter of external sCO₂ [m]
    DH₂ = ones(Int(Nⱼ),Int(Nₖ)) # Hydraulic diameter of internal sCO₂ [m]
    XMₛ = ones(Int(Nⱼ),Int(Nₖ)) # Porosity of solid [-]

    θ = 0.0
    Aₜₒₜ = 0.0
    Vₜₒₜ = ones(Int(NbT+2))
    AtotN = 0.0

    #Gridding
    Y = ones(Int(Nⱼ))
    Z = ones(Int(Nₖ))
    Hⱼ = ones(Int(Nⱼ))
    Hₖ = ones(Int(Nₖ))

    Y[1] = 0
    for j in 2:Int(Nⱼ)
        Y[j] = (Lⱼ/(Nⱼ-1)) * (j-1)
        Hⱼ[j-1] = Y[j]-Y[j-1]
    end
    Hⱼ[Int(Nⱼ)] = Hⱼ[Int(Nⱼ-1)]
    
    # Z-gridding later

    
    #Velocity, Reynolds number and friction factor calculation
    u₁ = ones(Int(Nⱼ),Int(Nₖ)) # Hot sCO₂ velocities (external flow) [m/s]
    u₂ = ones(Int(Nⱼ),Int(Nₖ)) # Cold sCO₂ velocities (in tubes) [m/s]
    Re₁ = ones(Int(Nⱼ),Int(Nₖ)) # Hot sCO₂ Reynold numbers (external flow) [-]
    Re₂ = ones(Int(Nⱼ),Int(Nₖ)) # Cold sCO₂ Reynold numbers (in tubes) [-]
    Cd₁ = ones(Int(Nⱼ),Int(Nₖ)) # Hot sCO₂ drag (external flow) [-]
    Cd₂ = ones(Int(Nⱼ),Int(Nₖ)) # Cold sCO₂ drag (in tubes) [-]
    Conv₁ = ones(Int(Nⱼ),Int(Nₖ)) # Convection heat generation (external flow) [W/m³/K]
    Conv₂ = ones(Int(Nⱼ),Int(Nₖ)) # Convection heat generation (internal flow) [W/m³/K]
    u₁ₘ = ones(Int(Nⱼ),Int(Nₖ)) # Maximum velocities of external flow
    
    T₁total = ones(Int(Nⱼ),Int(Nₖ*(NbT+2)),Int(Nₜᵢ)) # Temperature of fluid in shell [°C]
    T₂total = ones(Int(Nⱼ),Int(Nₖ*(NbT+2)),Int(Nₜᵢ)) # Temperature of fluid in tubes [°C]
    Tₛtotal = ones(Int(Nⱼ),Int(Nₖ*(NbT+2)),Int(Nₜᵢ)) # Temperature of solid [°C]
    Tₛₒtotal = ones(Int(Nⱼ),Int(Nₖ*(NbT+2)),Int(Nₜᵢ))
    Tₛᵢtotal = ones(Int(Nⱼ),Int(Nₖ*(NbT+2)),Int(Nₜᵢ))

    T₁ = ones(Int(Nⱼ),Int(Nₖ)) # Temperature of fluid in shell [°C]
    T₂ = ones(Int(Nⱼ),Int(Nₖ)) # Temperature of fluid in tubes [°C]
    Tₛ = ones(Int(Nⱼ),Int(Nₖ)) # Temperature of solid [°C]
    Tₛₒ = ones(Int(Nⱼ),Int(Nₖ))
    Tₛᵢ = ones(Int(Nⱼ),Int(Nₖ))
    Y₁ = ones(Int(Nⱼ),Int(Nₖ))
    Y₂ = ones(Int(Nⱼ),Int(Nₖ))
    Y₂ₒ = ones(Int(Nⱼ),Int(Nₖ))
    Y₂ᵢ = ones(Int(Nⱼ),Int(Nₖ))
    Y₃ = ones(Int(Nⱼ),Int(Nₖ))
    HH₁ = ones(Int(Nⱼ),Int(Nₖ)) # Heat transfer coefficient (external flow) [W/m²/K]
    HH₂ = ones(Int(Nⱼ),Int(Nₖ)) # Heat transfer coefficient (internal flow) [W/m²/K]
    CT₁ₛ = ones(Int(Nⱼ),Int(Nₖ)) # Volumetric heat generation between external fluid and solid [W/m³/K]
    CT₂ₛ = ones(Int(Nⱼ),Int(Nₖ)) # Volumetric heat generation between internal fluid and solid [W/m³/K]
    GTs = ones(Int(Nⱼ),Int(Nₖ))
    HTs = ones(Int(Nⱼ),Int(Nₖ))
    ITs = ones(Int(Nⱼ),Int(Nₖ))
    JTs = ones(Int(Nⱼ),Int(Nₖ))


    AJTₛ = ones(Int(Nⱼ),Int(Nₖ))
    BJTₛ = ones(Int(Nⱼ),Int(Nₖ))
    CJTₛ = ones(Int(Nⱼ),Int(Nₖ))
    AITₛ = ones(Int(Nⱼ),Int(Nₖ))
    BITₛ = ones(Int(Nⱼ),Int(Nₖ))
    CITₛ = ones(Int(Nⱼ),Int(Nₖ))
    CT₁ = ones(Int(Nⱼ),Int(Nₖ))
    CT₂ = ones(Int(Nⱼ),Int(Nₖ))
    CJTₛₛ = ones(Int(Nⱼ),Int(Nₖ))

    TInExt = ones(Int(NbT+2))
    TInInt = ones(Int(NbT+2))
    TOutExt = ones(Int(NbT+2))
    TOutInt = ones(Int(NbT+2))
    Tso = ones(Int(NbT+2))

    T1avgo = ones(Int(NbT+2))
    T1avgn = ones(Int(NbT+2))
    T2avgo = ones(Int(NbT+2))
    T2avgn = ones(Int(NbT+2))
    Tsoavg = ones(Int(NbT+2))

    T1ca = ones(Int(NbT+2), Nₖ)
    T2ca = ones(Int(NbT+2), Nⱼ)
    
    AHTBi = ones(Int(NbT+2))
    AcBi = ones(Int(NbT+2))
    DhBi = ones(Int(NbT+2))
    umax = ones(Int(NbT+2))
    ReBi = ones(Int(NbT+2))
    Dₑ = ones(Int(NbT+2))
    Nf = ones(Int(NbT+2))
    
    ηf = ones(Int(Nⱼ),Int(Nₖ))
    SW₁mod = ones(Int(Nⱼ),Int(Nₖ))
    
    mfint = 0.0
    SW₂mod = ones(Int(Nⱼ),Int(Nₖ))
    ηfint = ones(Int(Nⱼ),Int(Nₖ))
    
    mf = 0.0
    C2f = 0.0
    Numf1 = 0.0
    Numf2 = 0.0
    Denf1 = 0.0
    Denf2 = 0.0
    Pmin = 0.0
    Lccpf = 0.0

    ΔP₁ =0.0
    η₁ = 0.0
    F₁ = 0.0
    PP₁ =0.0
    ΔP₂ =0.0
    η₂ = 0.0
    F₂ = 0.0
    PP₂ =0.0
    PPt =0.0

    ΔP₁ₜ=0.0
    F₁ₜ =0.0
    PP₁ₜ=0.0
    ΔP₁w=0.0
    ΔPw =0.0

    ΔP₂ₜ=0.0
    F₂ₜ =0.0
    PP₂ₜ=0.0
    
    tempT1 = 0.0
    tempT2 = 0.0

#     uoav=0.0
#     uiav=0.0
#     foav=0.0
#     fiav=0.0
#     Dhoav=0.0
#     Dhiav=0.0
#     ρoav=0.0
#     ρiav=0.0

#     dP2 = 0.0
#     Ltemp = 0.0

    ϕₛ = ones(Int(NbT+2))
    ρi = ones(Int(NbT+2))
    ρo = ones(Int(NbT+2))
    ui = ones(Int(NbT+2))
    uo = ones(Int(NbT+2))
    fi = ones(Int(NbT+2))
    fo = ones(Int(NbT+2))
    Dhi = ones(Int(NbT+2))
    Dho = ones(Int(NbT+2))
    Rei = ones(Int(NbT+2))
    Reo = ones(Int(NbT+2))
    hiav = ones(Int(NbT+2))
    hoav = ones(Int(NbT+2))
    hi = ones(Int(Nⱼ))
    ho = ones(Int(Nₖ))
    hc1 = ones(Int(NbT+2))
    hc2 = ones(Int(NbT+2))
    Tsext = ones(Int(NbT+2))
    Tsavg = ones(Int(NbT+2))
    Tsint = ones(Int(NbT+2))
    ufr = ones(Int(NbT+2))
    tempTprev = 0.0

    h₁ᵢ = 0.0
    Cₚ₍₁ᵢ₎ = 0.0
    μ₁ᵢ = 0.0
    ρ₁ᵢ = 0.0
    ν₁ᵢ = 0.0
    κ₁ᵢ = 0.0
    α₁ᵢ = 0.0

    h₂ᵢ = 0.0
    Cₚ₍₂ᵢ₎ = 0.0
    μ₂ᵢ = 0.0
    ρ₂ᵢ = 0.0
    ν₂ᵢ = 0.0
    κ₂ᵢ = 0.0
    α₂ᵢ = 0.0

    Qⱼ₁ = 0.0
    Qₖ₁ = 0.0
    T₁₍ₐᵥ₎ = 0.0
    T₂₍ₐᵥ₎ = 0.0
    Qⱼ₂ = 0.0
    Qₖ₂ = 0.0
    ϵ = 0.0
    ϵ₁ = 0.0
    ϵ₂ = 0.0
    ϵₛ = 0.0
    Nu₁ = 0.0
    Nu₂ = 0.0
    HBₖ₂ = 0
    HBₖ₂ₘ = 0
    HBⱼ₂ = 0
    HBⱼ₂ₘ = 0
    
    Gtemp1 = 0.0
    Htemp1 = ones(Int(Nⱼ),Int(Nₖ))
    Itemp1 = 0.0
    Jtemp1 = ones(Int(Nⱼ),Int(Nₖ))
    GTso1 = ones(Int(Nⱼ),Int(Nₖ))
    ITso1 = ones(Int(Nⱼ),Int(Nₖ))
    
    ζl = ones(Int(NbT+2))
    ζb = ones(Int(NbT+2))
    As = ones(Int(NbT+2))
    ζs = 2
    Aw = 0.0
    
    DF = 0.0
    tubeP = ones(Int(NbT+2))
    windowP = ones(Int(NbT+1))
    
    ΔP₁tube = 0.0
    ΔP₁tube = ΔP₁ₜ
    
    ΔPtur = 0.0
    ΔPce = 0.0
    PPₜ = 0.0
    
    uturn = ones(Int(NbT+1))
    uhead = ones(Int(NbT+1))
    uheadPr = ones(Int(NbT+1))
    Pratio = ones(Int(NbT+1))
    PRAv = 0.0
    IntTubeP = 0.0
    
    DPInt = ones(Int(NbT+2))
    
    ΔPiu = ones(Int(Nₜᵢ)) # Pressure drop in each column of tubes for internal flow [Pa]
    ΔPiutot = 0.0 # Average pressure drop in u-bend region for internal flow [Pa]
    Rbend = ones(Int(Nₜᵢ)) # Radius of bend for each column of tube [m]
    kbend = ones(Int(Nₜᵢ)) # Bend factor for each row of tube [-]
    RbendoDtint = ones(Int(Nₜᵢ)) # Ratio of radius of bend to internal tube diameter [-]
    
    kbendxdata = [0.5,0.517,0.537,0.57,0.6,0.621,0.643,0.689,0.733,0.78,0.832,0.886,0.952,1.05,1.15,1.23,1.34,1.53,1.7,1.91,2.11,2.37,2.62,3.26,3.71,4.32,5.1,6.32,7.33,8.1,8.93,9.7,9.97,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,6000]
    kbendydata = [1.1,1.05,1.01,0.928,0.876,0.84,0.799,0.743,0.698,0.657,0.613,0.58,0.547,0.498,0.463,0.44,0.414,0.381,0.357,0.335,0.317,0.295,0.284,0.259,0.245,0.229,0.21,0.189,0.173,0.174,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173,0.173]
    kbenddata = interpolate(kbendxdata, kbendydata)
    
    DiffPtempInt = 100.0
    DiffPtempExt = 100.0

    u₁₍ₐᵥ₎ = 0.0
    u₂₍ₐᵥ₎ = 0.0

    Qext = 0.0
    Qint = 0.0
    FT₁ᵢ = 0.0
    
    Dotl = 0.0
    Acr = ones(Int(NbT+2))
    
    rs = 0.0
    rlm = ones(Int(NbT+2))
    pcorr = 0.0
    
    wp = 0.0
    Abp = ones(Int(NbT+2))
    
    Nss = Nss # Number of sealing strip pairs
    Ncc = 0
    
    rb = ones(Int(NbT+2))
    Nss⁺ = 0.0
    Dcorr = 3.7 # 4.5 for Re<100
    
    Fc = 0.0
    Ccorr = 1.25 # 1.35 for Re<100
    
    Li⁺ = 1 # (equation changes when baffle spacing varies)
    Lo⁺ = 1 # (equation changes when baffle spacing varies)
    ncorr = 0.6 # 0.33 for laminar flow
    
    Jc = 0.0
    Jl = ones(Int(NbT+2))
    Jb = ones(Int(NbT+2))
    Js = 0.0
    Jr = 0.0
    
    Jcorr = ones(Int(NbT+2))

    c=0

    P1tempO = P₁₍ᵢₙ₎
    P2tempO = P₂₍ᵢₙ₎
    
    Diff3 = 100.0

    Effo = 0.00
    Effn = 0.50
    Efft = 1.00
    
    while DiffPtempInt>0.01 || DiffPtempExt>0.01
        
        @label start
        while Diff3>0 || (-1*Diff3)>1
            
            Fh₁ᵢ = h1itemp - (Effn*Qmax/ṁₕ)
            FT₁ᵢ = PropsSI("T","H",Fh₁ᵢ,"P",P₁₍ᵢₙ₎,"CO2")-273
    
            
            for d in 1:NbT+2
                
                θ = 2*atand((Lⱼ/(2*Lₕ)))
                AtotN = ( ((θ*π*NDₛ*NDₛ)/(4*360)) + ((Lₕ*Lⱼ)/2) - (tₛ*Lⱼ) )
                
                if ExD < 0
                    Aₜₒₜ = ( ((θ*π*NDₛ*NDₛ)/(4*360)) + ((Lₕ*Lⱼ)/2) - (tₛ*Lⱼ) )
                    Vₜₒₜ[d] = Aₜₒₜ * Sₖ[d]
                else
                    Aₜₒₜ = Lᵢ*Lⱼ
                    Vₜₒₜ[d] = Aₜₒₜ * Sₖ[d]
                end
                
                if testPf == 1
                    Pf = Sₖ[d]
                    δfb = 0
                    Df = Dₜ
                end
                
                Nf[d] = round(Int64,(Sₖ[d]/Pf)) # Number of fins in one unit cell [-]
                
                if Nf[d]<1
                    Pf = Sₖ[d]
                    δfb = 0
                    Df = Dₜ
                    Nf[d]=1
                end
                
                Pmin = Pᵢ # minimum flow path [m]
                
                if FinType == 0 || FinType == 1
                    
                    for k in 1:Int(Nₖ)
                        for j in 1:Int(Nⱼ)
                            XM₁[j,k] = 1 - (((Nₜₜ*Sₖ[d]*π*Dₜ*Dₜ) + (Nf[d]*Nₜₜ*π*((Df*Df)-(Dₜ*Dₜ))*δfb))/ (4*Vₜₒₜ[d]))
                            XM₂[j,k] = (((Nₜₜ*π*Dᵢₚ*Dᵢₚ*Sₖ[d]) - (Nₜₜ*4*Ncruc*Ch*Ct*Sₖ[d]) - (Nₜₜ*4*Ct*Ct*Sₖ[d]))/(4*Vₜₒₜ[d]))
                            SW₁[j,k] = (((Nₜₜ*Nf[d]*2*π*Dₜ*(Pf-δfb)) + (Nₜₜ*Nf[d]*2*π*Df*δfb) + (Nₜₜ*Nf[d]*π*((Df*Df)-(Dₜ*Dₜ)))) / (2*Vₜₒₜ[d]))
                            SW₂[j,k] = ((Nₜₜ*π*Dᵢₚ*Sₖ[d]) - (Nₜₜ*Ncruc*Sₖ[d]*Ct) + (Nₜₜ*2*Ncruc*Ch*Sₖ[d])) / (Vₜₒₜ[d])
                            DH₁[j,k] = (4.0*XM₁[j,k])/SW₁[j,k]
                            DH₂[j,k] = (4.0*XM₂[j,k])/SW₂[j,k]
                            XMₛ[j,k] = 1.0 - (XM₁[j,k]+XM₂[j,k])
                       end
                    end
                    
                    ϕₛ[d] = XMₛ[1,1]
                    
                    AHTBi[d] = ((Nf[d]*π*Dₜ*(Pf-δfb)) + (Nf[d]*π*Df*δfb) + (Nf[d]*π*((Df*Df)-(Dₜ*Dₜ))/2)) # Area of heat transfer as derived by Biery [m²]
        
                    Pmin = Pᵢ # minimum flow path [m]
                    if (Pᵢ+Dₜ)<=(2*Pₓ)
                        Pmin = Pᵢ
                        AcBi[d] = ((Nf[d]*(Pmin-Dₜ)*Pf) - (Nf[d]*(Df-Dₜ)*δfb)) # Cross-section area as defined by Biery [m]
                    else
                        Pmin = Pₓ
                        AcBi[d] = 2*((Nf[d]*(Pmin-Dₜ)*Pf) - (Nf[d]*(Df-Dₜ)*δfb)) # Cross-section area as defined by Biery [m]
                    end
                    
                elseif FinType == 2
                    
                    for k in 1:Int(Nₖ)
                        for j in 1:Int(Nⱼ)
                            XM₁[j,k] = 1 - (Nₜₜ*((2*π*Dₜ*Dₜ*Sₖ[d]) + (Nf[d]*NAF*π*δfb*δfb*(Df-Dₜ)))/(8*Vₜₒₜ[d]))
                            XM₂[j,k] = (((Nₜₜ*π*Dᵢₚ*Dᵢₚ*Sₖ[d]) - (Nₜₜ*4*Ncruc*Ch*Ct*Sₖ[d]) - (Nₜₜ*4*Ct*Ct*Sₖ[d]))/(4*Vₜₒₜ[d]))
                            SW₁[j,k] = (((Nₜₜ*Nf[d]*2*π*Dₜ*Pf) + (Nₜₜ*Nf[d]*NAF*π*(Df-Dₜ)*δfb)) / (2*Vₜₒₜ[d]))
                            SW₂[j,k] = ((Nₜₜ*π*Dᵢₚ*Sₖ[d]) - (Nₜₜ*Ncruc*Sₖ[d]*Ct) + (Nₜₜ*2*Ncruc*Ch*Sₖ[d])) / (Vₜₒₜ[d])
                            DH₁[j,k] = (4.0*XM₁[j,k])/SW₁[j,k]
                            DH₂[j,k] = (4.0*XM₂[j,k])/SW₂[j,k]
                            XMₛ[j,k] = 1.0 - (XM₁[j,k]+XM₂[j,k])
                       end
                    end
                    
                    ϕₛ[d] = XMₛ[1,1]
                    
                    AHTBi[d] = ((Nf[d]*π*Dₜ*Pf) + (Nf[d]*NAF*π*(Df-Dₜ)*δfb)) # Area of heat transfer as derived by Biery [m²]
        
                    Pmin = Pᵢ # minimum flow path [m]
                    if (Pᵢ+Dₜ)<=(2*Pₓ)
                        Pmin = Pᵢ
                        AcBi[d] = ((Nf[d]*(Pmin-Dₜ)*Pf) - (Nf[d]*(Df-Dₜ)*δfb)) # Cross-section area as defined by Biery [m]
                    else
                        Pmin = Pₓ
                        AcBi[d] = 2*((Nf[d]*(Pmin-Dₜ)*Pf) - (Nf[d]*(Df-Dₜ)*δfb)) # Cross-section area as defined by Biery [m]
                    end
                    
                end
                
                DhBi[d] = (2*AcBi[d]*Pⱼ)/(AHTBi[d]) # Hydraulic diameter as defined by Biery [m]
                
                Dₑ[d] = sqrt(((Dₜ*Dₜ) + (((Df*Df)-(Dₜ*Dₜ))*δfb/Pf))) # Effective diameter as defined by Kaiyuan [m]
                
                Z[1] = 0
                for k in 2:Int(Nₖ)
                    Z[k] = (Sₖ[d]/(Nₖ-1)) * (k-1)
                    Hₖ[k-1] = Z[k]-Z[k-1]
                end
                Hₖ[Int(Nₖ)] = Hₖ[Int(Nₖ-1)]
                
                #Correction factor for HTC and ΔP in the shell-side
                Dotl = Dₛ-(2*Dₜ) # diameter of the circle through the ends of the outermost tubes [m]
                Acr[d] = (((Dₛ - Dotl)/2) + (((Dctl/2)-tₛ)*(Pᵢ-Dₜ)/Pᵢ) )*Sₖ[d] # flow area at or near the shell centerline for one crossflow section in a shell-and-tube exchanger [m²]
                
                rs = (Asb/(Asb+Atb)) # [-]
                rlm[d] = ((Asb+Atb)/Acr[d]) # [-]
                pcorr = ((-0.15*(1+rs))+0.8) # [-]
                
                wp = 0.75*Dₜ # distance between the splitter and the first tube [m]
                Abp[d] = (((Dₛ - Dotl)/2) +  (Np*wp/2)) * Sₖ[d] # flow bypass area of one baffle [m²]
                
                Nss = Nss # Number of sealing strip pairs
                # Ncc = (Dₛ - (2*lc))/(Pⱼ/2) # number of effective tube rows crossed during flow through one crossflow section (between baffle tips) [-]
                Ncc = Nₜⱼ # When no tubes in the window
                
                rb[d] = Abp[d]/Acr[d] # [-]
                Nss⁺ = Nss/Ncc # [-]
                Dcorr = 3.7 # 4.5 for Re<100
                
                Fc = 1 - (2*Fw)
                Ccorr = 1.25 # 1.35 for Re<100
                
                Li⁺ = 1 # (equation changes when baffle spacing varies)
                Lo⁺ = 1 # (equation changes when baffle spacing varies)
                ncorr = 0.6 # 0.33 for laminar flow
                
                # Jc = 0.55 + (0.72*Fc) # correction factor for baffle configuration
                Jc = 1 # When no tubes are in the window
                Jl[d] = ((0.44*(1-rs)) + ((1-(0.44*(1-rs)))*exp(-2.2*rlm[d]))) # correction factor for baffle leakage effects, including both tube-to-baffle and baffle- to-shell leakages
                Jb[d] = 1 # correction factor for bundle and pass partition bypass
                Js = ((NbT + NF - 1) + (Li⁺^(1-ncorr)) + (Lo⁺^(1-ncorr)))/((NbT + NF - 1) + (Li⁺) + (Lo⁺)) # correction factor for larger baffle spacing at the inlet and outlet sections compared to the central baffle spacing
                Jr = 1 # different for Re<100
                
                if Nss⁺<0.5
                    Jb[d] = exp((-Ccorr*rb[d]*(1-((2*Nss⁺)^(0.33))))) # correction factor for bypass flow
                else
                    Jb[d] = 1
                end
                
                if Bypass == 1
                    Jb[d] = 1
                end
                
                Jcorr[d] = Jc*Jl[d]*Jb[d]*Js*Jr # correction factor for shell-side heat transfer
    
                Diff1 = 100
                Diff2 = 100
    
                if d==1
                    TInExt[d] = FT₁ᵢ
                    TInInt[d] = T₂₍ᵢₙ₎
                    Tso[d] = (TInExt[d]+TInInt[d])/2
                else
                    TInExt[d] = TOutExt[d-1]
                    TInInt[d] = TOutInt[d-1]
                    Tso[d] = (TInExt[d]+TInInt[d])/2
                end
    
                if d==1
                    T1avgo[d] = FT₁ᵢ
                    T2avgo[d] = T₂₍ᵢₙ₎
                    T1avgn[d] = FT₁ᵢ
                    T2avgn[d] = T₂₍ᵢₙ₎
                    Tsoavg[d] = (TInExt[d]+TInInt[d])/2
                else
                    T1avgo[d] = TInExt[Int(d-1)]
                    T2avgo[d] = TInInt[Int(d-1)]
                    T1avgn[d] = TInExt[Int(d-1)]
                    T2avgn[d] = TInInt[Int(d-1)]
                    Tsoavg[d] = (TInExt[d]+TInInt[d])/2
                end
    
                T1temp = T1avgn[d]+273
                T2temp = T2avgn[d]+273
    
                iterno = 1
    
                while Diff1>1 || Diff2>1
    
                    h₁ᵢ = PropsSI("H","T",T1temp,"P",P₁[d],"CO2") # Initial Enthalpy of sCO₂ in the shell [J/kg]
                    Cₚ₍₁ᵢ₎ = PropsSI("C","T",T1temp,"P",P₁[d],"CO2") # Initial Specific heat of sCO₂ in the shell [J/kg/K]
                    μ₁ᵢ = PropsSI("V","T",T1temp,"P",P₁[d],"CO2") # Initial Dynamic viscosity of sCO₂ in the shell [Pa-s]
                    ρ₁ᵢ = PropsSI("D","T",T1temp,"P",P₁[d],"CO2") # Initial Density of sCO₂ in the shell [kg/m³]
                    ν₁ᵢ = μ₁ᵢ/ρ₁ᵢ # Initial Kinematic viscosity of sCO₂ in the shell [m²/s]
                    κ₁ᵢ = PropsSI("conductivity","T",T1temp,"P",P₁[d],"CO2") # Initial Thermal conductivity of sCO₂ in the shell [W/m/K]
                    α₁ᵢ = κ₁ᵢ/(ρ₁ᵢ*Cₚ₍₁ᵢ₎) # Initial Thermal diffusivity of sCO₂ in the shell [m²/s]
    
                    h₂ᵢ = PropsSI("H","T",T2temp,"P",P₂[d],"CO2")  # Initial Enthalpy of sCO₂ in the tubes [J/kg]
                    Cₚ₍₂ᵢ₎ = PropsSI("C","T",T2temp,"P",P₂[d],"CO2")  # Initial Specific heat of sCO₂ in the tubes [J/kg/K]
                    μ₂ᵢ = PropsSI("V","T",T2temp,"P",P₂[d],"CO2")  # Initial Dynamic viscosity of sCO₂ in the tubes [Pa-s]
                    ρ₂ᵢ = PropsSI("D","T",T2temp,"P",P₂[d],"CO2") # Initial Density of sCO₂ in the tubes [kg/m³]
                    ν₂ᵢ = μ₂ᵢ/ρ₂ᵢ # Initial Kinematic viscosity of sCO₂ in the tubes [m²/s]
                    κ₂ᵢ = PropsSI("conductivity","T",T2temp,"P",P₂[d],"CO2") # Initial Thermal conductivity of sCO₂ in the tubes [W/m/K]
                    α₂ᵢ = κ₂ᵢ/(ρ₂ᵢ*Cₚ₍₂ᵢ₎) # Initial Thermal diffusivity of sCO₂ in the tubes [m²/s]
    
                    if Tsoavg[d]>850
                        @goto eff
                    end
                    
                    Cₚ₍ₛ₎ = Cpsitp(Tsoavg[d]) # Specific heat of Haynes282
                    ρₛ = ρsitp(Tsoavg[d]) # Density of Haynes282
                    κₛ = κsitp(Tsoavg[d]) # Thermal conductivity of Haynes282
                    αₛ = αsitp(Tsoavg[d]) # Thermal diffusivity of Haynes282
    
                    u₁₍ₐᵥ₎ = 0.0
                    u₂₍ₐᵥ₎ = 0.0
    
                    
                    for k in 1:Int(Nₖ)
                        u₁[1,k] = ((ṁₕ)/(Lᵢ*Sₖ[d]*XM₁[1,k]*ρ₁ᵢ))
                        u₁₍ₐᵥ₎ = u₁₍ₐᵥ₎ + (u₁[1,k]*Hₖ[k])
                        Re₁[1,k] = abs((u₁[1,k]*DH₁[1,k]/ν₁ᵢ))
                    end
                    u₁₍ₐᵥ₎ = u₁₍ₐᵥ₎/Sₖ[d]
    
                    umax[d] = u₁[1,1]*DH₁[1,1]/DhBi[d] # Maximum velocity of external flow [m/s]
                    ReBi[d] = (umax[d]*DhBi[d])/ν₁ᵢ # Reynolds number as defined by Biery [-]
                    ufr[d] = ((ṁₕ)/(Lᵢ*Sₖ[d]*ρ₁ᵢ))
                    
                    for j in 1:Int(Nⱼ)
                        u₂[j,1] = ((ṁₗ)/(Lᵢ*Lⱼ*XM₂[j,1]*ρ₂ᵢ))
                        u₂₍ₐᵥ₎ = u₂₍ₐᵥ₎ + (u₂[j,1]*Hⱼ[j])
                        Re₂[j,1] = abs((u₂[j,1]*DH₂[j,1]/ν₂ᵢ))
                    end
                    u₂₍ₐᵥ₎ = u₂₍ₐᵥ₎/Lⱼ # Through each grid point
    
                    for k in 1:Int(Nₖ)
                        for j in 1:Int(Nⱼ)
                            u₁[j,k] = u₁[1,k] # [m/s] Uniform flow field
                            u₂[j,k] = u₂[j,1] # [m/s] Uniform flow field
                            Re₁[j,k] = Re₁[1,k] # [m/s] Uniform flow field
                            Re₂[j,k] = Re₂[j,1] # [m/s] Uniform flow field
    
                            if FinType == 0
                                Cd₁[j,k] = (0.3622 * ((DhBi[d]/Dₑ[d])^(0.7642)) * ((Pᵢ/Dₜ)^(0.2711)) * ((Pⱼ/(2*Dₜ))^(-0.4896)) * ((Re₁[1,1])^(-0.1801)) * 4 * DH₁[j,k] * DH₁[j,k] * DH₁[j,k]) / (DhBi[d] * DhBi[d] * DhBi[d]) # Friction factor for bare tubes [-]
                            elseif FinType == 1
                                Cd₁[j,k] = (0.5529 * ((DhBi[d]/Dₑ[d])^(0.894)) * ((Pᵢ/Dₜ)^(-0.0878)) * ((Pⱼ/(2*Dₜ))^(-0.5328)) * ((Df/Dₜ)^(0.1075)) * (((Pf-δfb)/Pf)^(-2.2775)) * ((Re₁[1,1])^(-0.2071)) * 4 * DH₁[j,k] * DH₁[j,k] * DH₁[j,k]) / (DhBi[d] * DhBi[d] * DhBi[d]) # Friction factor for disc fins [-]
                            elseif FinType == 2
                                Cd₁[j,k] = (0.2252 * ((DhBi[d]/Dₑ[d])^(0.3195)) * ((Pᵢ/Dₜ)^(1.1363)) * ((Pⱼ/(2*Dₜ))^(0.0696)) * ((Df/Dₜ)^(-0.8203)) * (((Pf-δfb)/Pf)^(0.2288)) * ((Re₁[1,1])^(-0.1976)) * 4 * DH₁[j,k] * DH₁[j,k] * DH₁[j,k]) / (DhBi[d] * DhBi[d] * DhBi[d]) # Friction factor for pin-fins [-]
                            end
                            
                            if (Re₂[j,k]) < 2300
                                Cd₂[j,k] = 64/(Re₂[j,k]) # Internal Drag
                            end
                            if (Re₂[j,k]) > 10000
                                FF = (((0.790*log(Re₂[j,k]))-1.64)^2)
                                Cd₂[j,k] = 1/(FF) # Internal Drag
                            end
                            if (Re₂[j,k]) > 2300 && (Re₂[j,k]) < 10000
                                AA = 64.0/2300.0
                                BB = 1.0/((((0.790*log(Re₂[j,k]))-1.64))^2)
                                Cd₂[j,k] = AA + ((BB-AA)*(Re₂[j,k]-2300)/(10000-2300)) # Internal Drag
                            end
    
                            Conv₁[j,k] = (ρ₁ᵢ*Cₚ₍₁ᵢ₎*u₁[j,k]*XM₁[j,k]/(Hⱼ[j])) # Refer closed VAT equations
                            Conv₂[j,k] = (ρ₂ᵢ*Cₚ₍₂ᵢ₎*u₂[j,k]*XM₂[j,k]/(Hₖ[k])) # Refer closed VAT equations
                        end
                    end
    
                    Qⱼ₁ = 0.0
                    Qₖ₁ = 0.0
                    T₁₍ₐᵥ₎ = 0.0
                    T₂₍ₐᵥ₎ = 0.0
                    Qⱼ₂ = 0.0
                    Qₖ₂ = 0.0
                    ϵ = 0.0
                    ϵ₁ = 0.0
                    ϵ₂ = 0.0
                    ϵₛ = 0.0
                    Nu₁ = 0.0
                    Nu₂ = 0.0
                    HBₖ₂ = 0
                    HBₖ₂ₘ = 0
                    HBⱼ₂ = 0
                    HBⱼ₂ₘ = 0
    
                    #Temperature field setup and thermal calculations
                    Pr₁ᵢ = ν₁ᵢ/α₁ᵢ # Prandtl number of external fluid [-]
                    Pr₂ᵢ = ν₂ᵢ/α₂ᵢ # Prandtl number of internal fluid [-]
                    
                    for k in 1:Int(Nₖ)
                        for j in 1:Int(Nⱼ)
                            T₁[j,k] = TInExt[d] # Temperature of fluid in shell [°C]
                            T₂[j,k] = TInInt[d] # Temperature of fluid in tubes [°C]
                            Tₛ[j,k] = (TInInt[d]+TInExt[d])/2.0 # Temperature of solid [°C]
                            Tₛₒ[j,k] = Tₛ[j,k] # Outer surface temperature [°C]
                            Tₛᵢ[j,k] = Tₛ[j,k] # Inner surface temperature [°C]
    
                            Y₁[j,k] = TInExt[d]
                            Y₂[j,k] = TInExt[d]
                            Y₃[j,k] = TInExt[d]
                            Y₂ₒ[j,k] = TInExt[d]
                            Y₂ᵢ[j,k] = TInExt[d]
    

                            if (Re₂[j,k]) < 2100
                                Nu₂ = 4.36 # Nusselt number of fluid in tubes [-]
                            else
                                Nu₂ = (Pr₂ᵢ*(Re₂[j,k]-1000)*(Cd₂[j,k]/8))/(1+(12.7*((Cd₂[j,k]/8)^(0.5))*((Pr₂ᵢ^(2/3))-1)))
                            end
                            
                            if FinType == 0
                                HH₁[j,k] = (0.3283 * ((DhBi[d]/Dₑ[d])^(0.4585)) * ((PᵢoDₜ)^(0.0739)) * ((PⱼoDₜ/2)^(-0.2187)) * ((ReBi[d])^(0.6111)) * κ₁ᵢ * (Pr₁ᵢ^(0.333))) / DhBi[d] # Heat transfer coefficient for bare tubes [W/m²/K]
                                HH₁[j,k] = HH₁[j,k]*Jcorr[d]
                                HH₂[j,k] = (Nu₂*κ₂ᵢ)/DH₂[1,1] # Heat transfer coefficient (internal flow) [W/m²/K]
                            elseif FinType == 1
                                HH₁[j,k] = (0.3574 * ((DhBi[d]/Dₑ[d])^(0.5029)) * ((PᵢoDₜ)^(-0.1789)) * ((PⱼoDₜ/2)^(-0.16)) * ((Df/Dₜ)^(-0.3304)) * (((Pf-δfb)/Pf)^(-2.3461)) * ((ReBi[d])^(0.6176)) * κ₁ᵢ * (Pr₁ᵢ^(0.333))) / DhBi[d] # Heat transfer coefficient for Disc fins [W/m²/K]
                                HH₁[j,k] = HH₁[j,k]*Jcorr[d]
                                HH₂[j,k] = (Nu₂*κ₂ᵢ)/DH₂[1,1] # Heat transfer coefficient (internal flow) [W/m²/K]
                            elseif FinType == 2
                                HH₁[j,k] = (0.3264 * ((DhBi[d]/Dₑ[d])^(0.3163)) * ((PᵢoDₜ)^(0.3123)) * ((PⱼoDₜ/2)^(-0.008)) * ((Df/Dₜ)^(0.0329)) * (((Pf-δfb)/Pf)^(0.1077)) * ((ReBi[d])^(0.5895)) * κ₁ᵢ * (Pr₁ᵢ^(0.333))) / DhBi[d] # Heat transfer coefficient for pin-fins [W/m²/K]
                                HH₁[j,k] = HH₁[j,k]*Jcorr[d]
                                HH₂[j,k] = (Nu₂*κ₂ᵢ)/DH₂[1,1] # Heat transfer coefficient (internal flow) [W/m²/K]
                            end
                            
                            
                            if FinType == 0 || FinType == 1
                                
                                if δfb>0
                                    mf = ((2*HH₁[j,k])/(κₛ*δfb))^(0.5)
                                    C2f = ((2*rₒₜ/mf) / ((rfc*rfc)-(rₒₜ*rₒₜ)))
                                    Numf1 = besselk(1,(mf*rₒₜ))*besseli(1,(mf*rfc))
                                    Numf2 = besseli(1,(mf*rₒₜ))*besselk(1,(mf*rfc))
                                    Denf1 = besseli(0,(mf*rₒₜ))*besselk(1,(mf*rfc))
                                    Denf2 = besselk(0,(mf*rₒₜ))*besseli(1,(mf*rfc))
                                    ηf[j,k] = C2f * ((Numf1-Numf2)/(Denf1+Denf2)) # Efficiency of a single disc fin [-]
                                else
                                    ηf[j,k]=0
                                end
                                
                                if Ct>0.0000001
                                    mfint = ((HH₂[j,k]*2*(Ct+Sₖ[d]))/(κₛ*Ct*Sₖ[d]))
                                    ηfint[j,k] = tanh(mfint*Ch)/(mfint*Ch)
                                else
                                    ηfint[j,k] = 0
                                    Ct = 0
                                end
                                
                                SW₁mod[j,k] = (((Nₜₜ*Nf[d]*2*π*Dₜ*(Pf-δfb)) + (Nₜₜ*Nf[d]*2*π*Df*δfb*ηf[j,k]) + (Nₜₜ*Nf[d]*π*((Df*Df)-(Dₜ*Dₜ))*ηf[j,k])) / (2*Vₜₒₜ[d])) # Modified specific wetted area for energy equation [1/m]
                                SW₂mod[j,k] = ((Nₜₜ*π*Dᵢₚ*Sₖ[d]) - (Nₜₜ*Ncruc*Sₖ[d]*Ct) + (Nₜₜ*2*Ncruc*Ch*Sₖ[d]*ηfint[j,k])) / (Vₜₒₜ[d])
                           
                            elseif FinType == 2
                                
                                if δfb>0
                                    mf = ((4*HH₁[j,k])/(κₛ*δfb))^(0.5)
                                    Lccpf = ((Df-Dₜ)/2) + (δfb/4)
                                    ηf[j,k] = tanh(mf*Lccpf)/(mf*Lccpf) # Efficiency of a single pin fin [-]
                                else
                                    ηf[j,k]=0
                                end
                                
                                if Ct>0.0000001
                                    mfint = ((HH₂[j,k]*2*(Ct+Sₖ[d]))/(κₛ*Ct*Sₖ[d]))
                                    ηfint[j,k] = tanh(mfint*Ch)/(mfint*Ch)
                                else
                                    ηfint[j,k] = 0
                                    Ct = 0
                                end
                                
                                SW₁mod[j,k] = (((Nₜₜ*Nf[d]*2*π*Dₜ*Pf) + (Nₜₜ*Nf[d]*NAF*π*(Df-Dₜ)*δfb*ηf[j,k])) / (2*Vₜₒₜ[d])) # Modified specific wetted area for energy equation [1/m]
                                SW₂mod[j,k] = ((Nₜₜ*π*Dᵢₚ*Sₖ[d]) - (Nₜₜ*Ncruc*Sₖ[d]*Ct) + (Nₜₜ*2*Ncruc*Ch*Sₖ[d]*ηfint[j,k])) / (Vₜₒₜ[d])
                                
                            end
                                
                            CT₁ₛ[j,k] = HH₁[j,k]*SW₁mod[j,k]
                            CT₂ₛ[j,k] = HH₂[j,k]*SW₂mod[j,k]
                            
                            Gtemp1 = ((HH₁[j,k]*0.5*Dₜ*log(Dₜ/Dint))/((HH₁[j,k]*0.5*Dₜ*log(Dₜ/Dint))+κₛ))-1
                            GTso1[j,k] = Gtemp1+1
                            Itemp1 = ((HH₂[j,k]*0.5*Dᵢₚ*log(Dint/Dᵢₚ))/((HH₂[j,k]*0.5*Dᵢₚ*log(Dint/Dᵢₚ))+κₛ))-1
                            ITso1[j,k] = Itemp1+1
                            GTs[j,k] = CT₁ₛ[j,k]*Gtemp1
                            ITs[j,k] = CT₂ₛ[j,k]*Itemp1
                            
                            Htemp1[j,k] = κₛ/((HH₁[j,k]*0.5*Dₜ*log(Dₜ/Dint))+κₛ)
                            Jtemp1[j,k] = κₛ/((HH₂[j,k]*0.5*Dᵢₚ*log(Dint/Dᵢₚ))+κₛ)
                            HTs[j,k] = CT₁ₛ[j,k]*Htemp1[j,k]
                            JTs[j,k] = CT₂ₛ[j,k]*Jtemp1[j,k]
                        end
                    end
                    
                    for k in 1:Int(Nₖ)
                        for j in 1:Int(Nⱼ)
    
                            κₛₖ = κₛ
    
                            if k == 1
                                HBₖ₂ = Hₖ[1]^2
                                HBₖ₂ₘ = HBₖ₂
                            else
                                HBₖ₂ₘ = (Hₖ[k-1]*((Hₖ[k-1]+Hₖ[k])/2.0))
                                HBₖ₂ = (Hₖ[k]*((Hₖ[k-1]+Hₖ[k])/2.0))
                            end
    
                            κₛⱼ = κₛ
    
                            if j == 1
                                HBⱼ₂ = Hⱼ[1]^2
                                HBⱼ₂ₘ = HBⱼ₂
                            else
                                HBⱼ₂ₘ = (Hⱼ[j-1]*((Hⱼ[j-1]+Hⱼ[j])/2.0))
                                HBⱼ₂ = (Hⱼ[j]*((Hⱼ[j-1]+Hⱼ[j])/2.0))
                            end
    
                            AJTₛ[j,k] = (XMₛ[j,k]*κₛₖ)/HBₖ₂ₘ
                            BJTₛ[j,k] = (XMₛ[j,k]*κₛₖ)/HBₖ₂
                            CJTₛ[j,k] = AJTₛ[j,k] + BJTₛ[j,k]
                            AITₛ[j,k] = (XMₛ[j,k]*κₛⱼ)/HBⱼ₂ₘ
                            BITₛ[j,k] = (XMₛ[j,k]*κₛⱼ)/HBⱼ₂
                            CITₛ[j,k] = AITₛ[j,k] + BITₛ[j,k]
    
                            CT₁[j,k] = Conv₁[j,k] + CT₁ₛ[j,k]
                            CT₂[j,k] = Conv₂[j,k] + CT₂ₛ[j,k]
                            
                            CJTₛₛ[j,k] = (CJTₛ[j,k]+HTs[j,k]+JTs[j,k])
                        end
                    end
    
    
                # Iterative procedure to obtain convergence
                    for m in 1:10000
    
                        #Solid side temperature
                        for k in 1:Int(Nₖ)
                            for j in 1:Int(Nⱼ)
    
                                if j == Int(Nⱼ)
                                    Tₛₚ = Tₛ[j,k]
                                else
                                    Tₛₚ = Tₛ[(j+1),k]
                                end
    
                                if j == 1
                                    Tₛₘ = Tₛ[j,k]
                                else
                                    Tₛₘ = Tₛ[(j-1),k]
                                end
    
                                if k == Int(Nₖ)
                                    Tₛⱼₚ = Tₛ[j,k]
                                else
                                    Tₛⱼₚ = Tₛ[j,(k+1)]
                                end
    
                                if k == 1
                                    Tₛⱼₘ = Tₛ[j,k]
                                else
                                    Tₛⱼₘ = Tₛ[j,(k-1)]
                                end
                                
                                Y₂[j,k] = (((AJTₛ[j,k]*Tₛⱼₘ)+(BJTₛ[j,k]*Tₛⱼₚ))/CJTₛₛ[j,k]) - (GTs[j,k]*T₁[j,k]/CJTₛₛ[j,k]) - (ITs[j,k]*T₂[j,k]/CJTₛₛ[j,k])
                                
                            end
                        end
    
    
                        #External flow temperature
                        for k in 1:Int(Nₖ)
                            for j in 1:Int(Nⱼ)
                                Y₂ₒ[j,k] = (GTso1[j,k]*T₁[j,k])+(Htemp1[j,k]*Y₂[j,k])
                                if j == 1
                                    Y₁[j,k] = TInExt[d]
                                else
                                    Y₁[j,k] = (Y₁[(j-1),k]-(CT₁ₛ[j,k]*Y₂ₒ[j,k]/Conv₁[j,k]))/(1.0-(CT₁ₛ[j,k]/Conv₁[j,k]))
                                end
                            end
                        end
    
    
                        #Internal flow temperature
                        for j in 1:Int(Nⱼ)
                            for k in 1:Int(Nₖ)
                                Y₂ᵢ[j,k] = (ITso1[j,k]*T₂[j,k])+(Jtemp1[j,k]*Y₂[j,k])
                                if k == 1
                                    Y₃[j,k] = TInInt[d]
                                else
                                    Y₃[j,k] = ((CT₂ₛ[j,k]*Y₂ᵢ[j,k]/Conv₂[j,k])+Y₃[j,(k-1)])/(1.0+(CT₂ₛ[j,k]/Conv₂[j,k]))
                                end
                            end
                        end
                        
                        #Error calculation
                        ϵ = 0
                        for k in 2:Int(Nₖ-1)
                            for j in 2:Int(Nⱼ-1)
                                ϵ₁ = abs((T₁[j,k]-Y₁[j,k])/T₁[j,k])
                                ϵₛ = abs((Tₛ[j,k]-Y₂[j,k])/Tₛ[j,k])
                                ϵₛₒ = abs((Tₛₒ[j,k]-Y₂ₒ[j,k])/Tₛₒ[j,k])
                                ϵₛᵢ = abs((Tₛᵢ[j,k]-Y₂ᵢ[j,k])/Tₛᵢ[j,k])
                                ϵ₂ = abs((T₂[j,k]-Y₃[j,k])/T₂[j,k])
    
                                if ϵ < ϵ₁
                                    ϵ = ϵ₁
                                end
                                if ϵ < ϵₛ
                                    ϵ = ϵₛ
                                end
                                if ϵ < ϵₛₒ
                                    ϵ = ϵₛₒ
                                end
                                if ϵ < ϵₛᵢ
                                    ϵ = ϵₛᵢ
                                end
                                if ϵ < ϵ₂
                                    ϵ = ϵ₂
                                end
                            end
                        end
    
    
                        #Temperature field updation
                        for k in 1:Int(Nₖ)
                            for j in 1:Int(Nⱼ)
                                T₁[j,k] = Y₁[j,k]
                                Tₛ[j,k] = Y₂[j,k]
                                Tₛₒ[j,k] = Y₂ₒ[j,k]
                                Tₛᵢ[j,k] = Y₂ᵢ[j,k]
                                T₂[j,k] = Y₃[j,k]
                            end
                        end
    
                        if((m+1) > 10) && (ϵ < 0.1*(10^-5))
                            break #Exit clause
                        end
    
                    end
                    
                    tempT1 = TInExt[1]
    
                    tempT2 = TInExt[1]
                    
                    for j in 1:Nⱼ
                        for k in 1:Nₖ
                            if T₁[j,k]<0 || T₂[j,k]<0
                                @goto eff
                            end
                        end
                    end
                    
                    for j in 1:Nⱼ
                        for k in 1:Nₖ
                            if isnan(T₁[j,k]) || isnan(T₂[j,k]) || isnan(-T₁[j,k]) || isnan(-T₂[j,k])
                                @goto eff
                            end
                        end
                    end
    
                    iterno = iterno+1
    
                    hiav[d] = 0.0
                    hi[1] = PropsSI("H","T",(T₂[1,Nₖ]+273),"P",P₂[d],"CO2")
                    for a in 2:Nⱼ
                        hi[a] = PropsSI("H","T",(T₂[a,Nₖ]+273),"P",P₂[d],"CO2")
                        hiav[d] = (((hi[a]+hi[a-1])/2)*(Lⱼ/Nⱼ)) + hiav[d]
                    end
                    hiav[d] = hiav[d]/(Lⱼ-(Lⱼ/Nⱼ))
    
                    hoav[d] = 0.0
                    ho[1] = PropsSI("H","T",(T₁[Nⱼ,1]+273),"P",P₁[d],"CO2")
                    for a in 2:Nₖ
                        ho[a] = PropsSI("H","T",(T₁[Nⱼ,a]+273),"P",P₁[d],"CO2")
                        hoav[d] = (((ho[a]+ho[a-1])/2)*(Sₖ[d]/Nₖ)) + hoav[d]
                    end
                    hoav[d] = hoav[d]/(Sₖ[d]-(Sₖ[d]/Nₖ))
    
                    if hiav[d]>3982400 || hoav[d]>3982400
                        @goto eff
                    end
    
                    TOutExt[d] = PropsSI("T","H",hoav[d],"P",P₁[d],"CO2")-273
                    TOutInt[d] = PropsSI("T","H",hiav[d],"P",P₂[d],"CO2")-273
    
                    T1avgo[d] = T1avgn[d]
                    T1avgn[d] = (TInExt[d]+TOutExt[d])/2
                    T2avgo[d] = T2avgn[d]
                    T2avgn[d] = (TInInt[d]+TOutInt[d])/2
                    Tsoavg[d] = (T1avgn[d]+T2avgn[d])/2
                    Tsext[d] = Tₛₒ[1,1]
                    Tsavg[d] = Tₛ[1,1]
                    Tsint[d] = Tₛᵢ[1,1]
                    
                    if Tsoavg[d]>850
                        @goto eff
                    end
                    
                    Diff1 = abs(T1avgn[d]-T1avgo[d])
                    Diff2 = abs(T2avgn[d]-T2avgo[d])
                    
                    T1temp = T1avgn[d]+273
                    T2temp = T2avgn[d]+273
                end
    
                for k in 1:Nₖ
                    T1ca[d,k] = T₁[Nⱼ,k]
                end
                for j in 1:Nⱼ
                    T2ca[d,j] = T₂[j,Nₖ]
                end
                
                for k in 1:Nₖ
                    for j in 1:Nⱼ
                        T₁total[j,Int(((d-1)*Nₖ)+k),1] = T₁[j,k] # Temperature of fluid in shell [°C]
                        T₂total[j,Int(((d-1)*Nₖ)+k),1] = T₂[j,k] # Temperature of fluid in tubes [°C]
                        Tₛtotal[j,Int(((d-1)*Nₖ)+k),1] = Tₛ[j,k] # Temperature of solid [°C]
                        Tₛₒtotal[j,Int(((d-1)*Nₖ)+k),1] = Tₛₒ[j,k]
                        Tₛᵢtotal[j,Int(((d-1)*Nₖ)+k),1] = Tₛᵢ[j,k]
                    end
                end
                
                ρi[d] = ρ₂ᵢ
                ρo[d] = ρ₁ᵢ
                ui[d] = u₂₍ₐᵥ₎
                uo[d] = u₁₍ₐᵥ₎
                fi[d] = Cd₂[1,1]
                fo[d] = Cd₁[1,1]
                Dhi[d] = DH₂[1,1]
                Dho[d] = DH₁[1,1]
                Rei[d] = Re₂[1,1]
                Reo[d] = Re₁[1,1]
                hc1[d] = HH₁[1,1]
                hc2[d] = HH₂[1,1]
            end
    
    
    
            if((TOutExt[(NbT+2)])<T₁₍ᵢₙ₎)
                Efft = Effn
                Effn = (Effo+Efft)/2
            elseif((TOutExt[(NbT+2)])>(T₁₍ᵢₙ₎+1))
                Effo = Effn
                Effn = (Effo+Efft)/2
            end
            
            if Effn>0.99
                Diff3 = 0.0
            end
            
            Diff3 = (T₁₍ᵢₙ₎ - TOutExt[(NbT+2)])
            
            if (Diff3-tempTprev)<0.000001 && (Diff3-tempTprev)>(-0.000001)
                Diff3 = 0.0
            end
            
            tempTprev = Diff3
    
            @goto start
    
            @label eff
    
            Effo = Effn
            Effn = (Effo+Efft)/2
            
        end
        
        @label stop
        
        for l in 1:Int(Nₜᵢ)
            for d in 1:Int(NbT+2)
                for k in 1:Nₖ
                    for j in 1:Nⱼ
                        T₁total[j,Int(((d-1)*Nₖ)+k),l] = T₁total[j,Int(((d-1)*Nₖ)+k),1] # Temperature of fluid in shell [°C]
                        T₂total[j,Int(((d-1)*Nₖ)+k),l] = T₂total[j,Int(((d-1)*Nₖ)+k),1] # Temperature of fluid in tubes [°C]
                        Tₛtotal[j,Int(((d-1)*Nₖ)+k),l] = Tₛtotal[j,Int(((d-1)*Nₖ)+k),1] # Temperature of solid [°C]
                        Tₛₒtotal[j,Int(((d-1)*Nₖ)+k),l] = Tₛₒtotal[j,Int(((d-1)*Nₖ)+k),1]
                        Tₛᵢtotal[j,Int(((d-1)*Nₖ)+k),l] = Tₛᵢtotal[j,Int(((d-1)*Nₖ)+k),1]
                    end
                end
            end
        end
        
        #Pressure drop and Pumping Power calculation
        
        a=1
        b=1
        
        ζs = 2
        Aw = 0.0
        ΔP₁ₜ = 0.0
        ΔP₂ₜ = 0.0
        
        for dtemp in 1:NbT+2
            ζl[dtemp] = exp((-1.33*(1+rs)*((rlm[dtemp])^(pcorr)))) # Correction factor for tube-to-baffle and shell-to-baffle leakage
            ζb[dtemp] = 1 # correction factor for bypass flow
            ζs = 2 # correction factor for inlet and outlet losses (equation changes when baffle spacing varies)
            
            if Nss⁺<0.5
                ζb[dtemp] = exp((-Dcorr*rb[dtemp]*(1-((2*Nss⁺)^(0.33))))) # correction factor for bypass flow
            else
                ζb[dtemp] = 1
            end
            
            As[dtemp] = (((Dₛ/2)-tₛ)*Sₖ[dtemp]*(1-(Dₜ/Pᵢ))) # Area at the centerline of the shell [m²]
            Aw = ((π*NDₛ*NDₛ/8) - (AtotN) - (tₛ*NDₛ))/2 # Area of the window for external flow turning [m²]
            
            if Bypass == 1
                ζb[dtemp] = 1 # correction factor for bypass flow
            end
        end
    
        DF = 0.0
        tubeP = ones(Int(NbT+2))
        windowP = ones(Int(NbT+1))
        
        for a in 1:Int(NbT+2)
            if (a == 1) || (a == (NbT+2))
                ΔP₁ = 1.0 * (((fo[a]*ρo[a]*(uo[a]^2)*Lⱼ) / (Dho[a]*2.0))) *(1+(Ncw/Ncc))*ζb[a]*ζs*ζl[a] # Entrance and exit pressure drop in external flow [N/m²]
            else
                ΔP₁ = ((fo[a]*ρo[a]*(uo[a]^2)*Lⱼ) / (Dho[a]*2.0))*ζb[a]*ζl[a] # pressure drop in center [N/m²]
            end
            η₁ = 1.0 # Pump efficiency [-]
            F₁ = (ΔP₁*Lᵢ*Sₖ[a]*XM₁[1,1]) # [N]
            PP₁ = ((F₁*uo[a])/η₁) # [W]
            ΔP₁ₜ = ΔP₁ₜ+ΔP₁
            F₁ₜ = F₁ₜ + F₁
            PP₁ₜ = PP₁ₜ + PP₁
            DF = (((fo[b]*ρo[b]*(uo[b]^2)*π*Dₜ*Sₖ[b]))/2.0)+DF
            tubeP[a] = ΔP₁
        end
        
        ΔP₁tube = 0.0
        ΔP₁tube = ΔP₁ₜ
        
        for a in 1:(NbT+1)
            ΔPw = ((ṁₕ*ṁₕ*(2 + (0.6*Ncw)))/(2*ρo[a]*As[a]*Aw))*ζl[a] # Pressure drop in the window for external flow [N/m²]
            η₁ = 1.0 # Pump efficiency [-]
            F₁ = (ΔPw*Lᵢ*Sₖ[a]*XM₁[1,1]) # [N]
            PP₁ = ((F₁*uo[a])/η₁) # [W]
            ΔP₁w = ΔP₁w+ΔPw
            ΔP₁ₜ = ΔP₁ₜ+ΔPw
            F₁ₜ = F₁ₜ + F₁
            PP₁ₜ = PP₁ₜ + PP₁
            windowP[a] = ΔPw
        end
        
        
        for b in 1:Int(NbT+2)
            ΔP₂ = (((fi[b]*ρi[b]*(ui[b]^2)*(Sₖ[b])/(Dhi[b]*2.0))))
            η₂ = 1.0 # Pump efficiency [-]
            F₂ = (ΔP₂*Lᵢ*Lⱼ*XM₂[1,1]) # [N]
            PP₂ = ((F₂*u₂[1,1])/η₂) # [W]
            ΔP₂ₜ = ΔP₂ₜ+ΔP₂
            F₂ₜ = F₂ₜ + F₂
            PP₂ₜ = PP₂ₜ + PP₂
            DPInt[b] = ΔP₂
        end
        
        DFO = DF/(NbT+2)
        
        Dexp = 1.8*(min(Pᵢ,Pⱼ,Pₓ)-(Dₜ/4)) # Diameter of expansion (imaginary circle)
        σ = (Dhi[1]*Dhi[1])/(Dexp*Dexp) # Ratio of area of cotracted to expanded tube
        Ke = ((1-σ)^(2))
        Kc = (0.5*(1-σ))
        Gt = ṁₕ/(Nₜₜ*π*Dhi[1]*Dhi[1]/4)
        
        ΔPce = (0.5*(NF+1)*(Kc+Ke)*Gt*Gt)/(2*((ρi[1]+ρi[(NbT+2)])/2)) # (NF+1)/2 instead of NF+1 for U-tubes
        
        ΔPiutot = 0.0 # Average pressure drop in u-bend region for internal flow [Pa]
        
        ρibend = (ρi[NbL+1]+ρi[NbL+2])/2
        uibend = (ui[NbL+1]+ui[NbL+2])/2
        fibend = (fi[NbL+1]+fi[NbL+2])/2
        
        c = 0
        θbend = 180 # bending angle of tube [in degrees]
        
        for c in 1:Nₜᵢ
            Rbend[c] = tₛ + (c*Pᵢ/2)
            RbendoDtint[c] = Rbend[c]/Dᵢₚ
            kbend[c] = kbenddata(RbendoDtint[c])
            ΔPiu[c] = ρibend*(uibend^2)*0.5*((fibend*π*Rbend[c]*θbend/(Dᵢₚ*180))+kbend[c])
            ΔPiutot = ΔPiutot + ΔPiu[c]
        end
        
        ΔPiutot = ΔPiutot/Nₜᵢ
        
        ΔP₂ₜ = ΔP₂ₜ + ΔPce + ΔPiutot # Total internal pressure drop [N/m²]
        
        PPₜ = PP₁ₜ + (PP₂ₜ*Nₜₜ)
        
        P1temp1 = P₁₍ᵢₙ₎
        P2temp1 = P₂₍ᵢₙ₎
        P1tempF = 0.0
        P2tempF = 0.0
        for a in 1:NbT+2
            if a == 1
                P₁[a] = P1temp1 - ((tubeP[a]+(windowP[a]/2))/2)
                P1temp1 = P1temp1 - ((tubeP[a]+(windowP[a]/2)))
            elseif a == (NbT+2)
                P₁[a] = P1temp1 - ((tubeP[a]+(windowP[a-1]/2))/2)
                P1temp1 = P1temp1 - ((tubeP[a]+(windowP[a-1]/2)))
            else
                P₁[a] = P1temp1 - ((tubeP[a]+(windowP[a-1]/2)+(windowP[a]/2))/2)
                P1temp1 = P1temp1 - ((tubeP[a]+(windowP[a-1]/2)+(windowP[a]/2)))
            end
            
            if a == 1
                P₂[a] = P2temp1 - (((DPInt[a]/2)+(ΔPce/2)))
                P2temp1 = P2temp1 - (DPInt[a]+(ΔPce/2))
            elseif a == (NbL+2)
                P₂[a] = P2temp1 - ((DPInt[a]/2)+(ΔPiutot))
                P2temp1 = P2temp1 - (DPInt[a]+(ΔPiutot))
            else
                P₂[a] = P2temp1 - ((DPInt[a]/2))
                P2temp1 = P2temp1 - (DPInt[a])
            end
        end
        P1tempF = P1temp1
        P2tempF = P2temp1 - (ΔPce/2)
        
        DiffPtempInt = abs((P2tempF-P2tempO)/P2tempO)
        DiffPtempExt = abs((P1tempF-P1tempO)/P1tempO)
        
        P1tempO = P1tempF
        P2tempO = P2tempF
        
        Diff3 = 100.0
        
        Effo = 0.00
        Effn = Effn
        Efft = 1.00
        
        if (ΔP₁ₜ>=P₁₍ᵢₙ₎) || (ΔP₂ₜ>=P₂₍ᵢₙ₎)
            @goto pressexcess
        end
        
    end
    
    
    PRAv = 0.0
    IntTubeP = 0.0
    
    for dtemp in 1:NbT+1
        Aturn = ((π*NDₛ*NDₛ/8) - (AtotN) - (tₛ*NDₛ))/2
        uturn[dtemp] = ṁₕ/(ρo[dtemp]*Aturn)
        uhead[dtemp] = (uturn[dtemp]*uturn[dtemp])/(2*9.81)
        uheadPr[dtemp] = (ρo[dtemp]*uturn[dtemp]*uturn[dtemp])/2
        if dtemp != 1
            IntTubeP = ((fo[dtemp]*ρo[dtemp]*(uo[dtemp]^2)*Lⱼ) / (Dho[dtemp]*2.0))*ζb[dtemp]*ζl[dtemp]
        else
            IntTubeP = ((fo[dtemp]*ρo[dtemp]*(uo[dtemp]^2)*Lⱼ) / (Dho[dtemp]*2.0))
        end
        Pratio[dtemp] = IntTubeP/uheadPr[dtemp]
        PRAv = PRAv + Pratio[dtemp]
    end
    
    PRAv = PRAv/(NbT+1)

    #Calculate heat exchanger effectiveness

    heia = PropsSI("H","T",(TOutExt[NbT+2]+273),"P",P₁₍ᵢₙ₎,"CO2")
    heoa = PropsSI("H","T",(TInExt[1]+273),"P",P₁₍ᵢₙ₎,"CO2")
    heii = PropsSI("H","T",(T₁₍ᵢₙ₎+273),"P",P₁₍ᵢₙ₎,"CO2")
    heoi = PropsSI("H","T",(T₂₍ᵢₙ₎+273),"P",P₁₍ᵢₙ₎,"CO2")
    Qae = ṁₕ*(heia-heoa)
    Qₘe = ṁₕ*(heii-heoi)

    hiia = PropsSI("H","T",(TInInt[1]+273),"P",P₂₍ᵢₙ₎,"CO2")
    hioa = PropsSI("H","T",(TOutInt[NbT+2]+273),"P",P₂₍ᵢₙ₎,"CO2")
    hiii = PropsSI("H","T",(T₁₍ᵢₙ₎+273),"P",P₂₍ᵢₙ₎,"CO2")
    hioi = PropsSI("H","T",(T₂₍ᵢₙ₎+273),"P",P₂₍ᵢₙ₎,"CO2")
    Qai = ṁₗ*(hioa-hiia)
    Qₘi = ṁₗ*(hiii-hioi)

    Qₘ = min(Qₘe,Qₘi)

    HXEffExt = Qae/Qₘ
    HXEffInt = Qai/Qₘ

    #Calculate heat transfer coefficient and Reynolds number
    Reintav = 0.0
    Reextav = 0.0
    hintav = 0.0
    hextav = 0.0
    ufront = 0.0
    uinternal = 0.0
    umaximum = 0.0
    
    for d in 1:(NbT+2)
        Reintav = Reintav + Rei[d]
        Reextav = Reextav + Reo[d]
        hintav = hintav + hc2[d]
        hextav = hextav + hc1[d]
        ufront = ufront + ufr[d]
        uinternal = uinternal + ui[d]
        umaximum = umaximum + umax[d]
    end
    
    Reintav = Reintav/(NbT+2)
    Reextav = Reextav/(NbT+2)
    hintav = hintav/(NbT+2)
    hextav = hextav/(NbT+2)
    ufront = ufront/(NbT+2)
    uinternal = uinternal/(NbT+2)
    umaximum = umaximum/(NbT+2)

    #Calculate Volume and weight
    Volume = 0.0
    NfinsL = 0
    NfinsR = 0
    ρₛ = ρsitp(550)
    for dtemp in 1:NbL+1
        NfinsL = NfinsL + Nf[dtemp]
    end
    for dtemp in NbL+2:NbT+2
        NfinsR = NfinsR + Nf[dtemp]
    end
    NfinsL = round(Int64,NfinsL)
    NfinsR = round(Int64,NfinsR)
    Volume = Volume + (NbT*((0.5*π*((NDₛ^2)-(Nₜₜ*(Dₜ^2)))/4)-(tₛ*NDₛ))*tₖ) + (π*((Dₒₛ^2)-(NDₛ^2))*Lₖ/4) + (tₛ*(NF+1)*NDₛ*Lₖ) + (2*Hₜ*π*((Dₒₛ^2)-(Nₜₜ*(Dᵢₚ^2)))/4) + ((NF)*NfinsL*Nₜₜ*π*((Df*Df)-(Dₜ*Dₜ))*δfb/4)+ ((NF)*NfinsR*Nₜₜ*π*((Df*Df)-(Dₜ*Dₜ))*δfb/4) + ((NF+1)*Nₜₜ*π*((Dₜ*Dₜ)-(Dᵢₚ*Dᵢₚ))*Lₖ/4) + ((NF+1)*Nₜₜ*((Ncruc*Ct*Ch*Lₖ) + (Ct*Ct*Lₖ))) + (((2*ExD)+(2*Lⱼ))*tsb*Lₖ)
    Weight = Volume*ρₛ
    HXVolume = (π*Dₒₛ*Dₒₛ*(Lₖ+(2*Hₜ)))/4
    
    print("Internal pressure drop                   : ",round((ΔP₂ₜ/100000),digits=3)," bar \n")
    print("External pressure drop                   : ",round((ΔP₁ₜ/100000),digits=3)," bar \n\n")
    print("Pumping power (Internal flow)            : ",round((PP₂ₜ*Nₜₜ),digits=3)," Watts \n")
    print("Pumping power (External flow)            : ",round((PP₁ₜ),digits=3)," Watts \n")
    print("Total pumping power                      : ",round((PPₜ),digits=3)," Watts \n\n")
    
    print("Total power (External)                   : ",round((Qae),digits=3)," Watts \n")
    print("Total power (Internal)                   : ",round((Qai),digits=3)," Watts \n")
    print("Maximum available power                  : ",round((Qₘ),digits=3)," Watts \n\n")
    
    print("Heat exchanger effectiveness (External)  : ", round(HXEffExt, digits=3) ,"\n")
    print("Heat exchanger effectiveness (Internal)  : ", round(HXEffInt, digits=3) ,"\n\n")
    
    print("External flow inlet temperature  [°C]    : ",round(TOutExt[NbT+2], digits=2),"\n")
    print("External flow outlet temperature [°C]    : ",round(TInExt[1], digits=2),"\n")
    print("Internal flow inlet temperature  [°C]    : ",round(TInInt[1], digits=2),"\n")
    print("Internal flow outlet temperature [°C]    : ",round(TOutInt[NbT+2], digits=2),"\n\n")
    
    print("HX Length                        [m]     : ",round(Lₖ, digits=6),"\n")
    print("Solid volume                     [m³]    : ",round(Volume, digits=6),"\n")
    print("HX Weight                        [kg]    : ",round(Weight, digits=4),"\n")
    print("HX Volume                        [m³]    : ",round(HXVolume, digits=6),"\n\n")
    
    output = open("HXTestData.csv", "a")
    CSV.File(output; header=false)
    CSV.File(output; dateformat="mm/dd/yyyy")
    Time = Dates.now()
    Time = Dates.format(Time, "e, dd u yyyy HH:MM:SS")
    OutParameters = [Int(counter) Time round(TOutExt[NbT+2],digits=2) round(TInInt[1],digits=2) round(TInExt[1],digits=2) round(TOutInt[NbT+2],digits=2) round(HXEffExt,digits=3) round(HXEffInt,digits=3) round(NDₛ,digits=4) round(Dₒₛ,digits=4) round(Dᵢₚ,digits=6) round(Dₜ,digits=6) round(ExD,digits=4) round(Nₜₜ*2,digits=2) round(NbT,digits=2) round(tₖ,digits=6) round(Nₜⱼ,digits=2) round(Nₜᵢ,digits=2) round(Pⱼ,digits=6) round(Pᵢ,digits=6) round(Lₖ,digits=3) round(Df,digits=6) round(δfb,digits=8) round(δft,digits=8) round(Pf,digits=8) round(Ct,digits=4) round(Reintav,digits=2) round(Reextav,digits=2) round(ufront,digits=4) round(uinternal,digits=4) round(hintav,digits=2) round(hextav,digits=2) round((ΔP₁w/100000),digits=3) round((ΔP₁tube/100000),digits=3) round(PRAv,digits=3) round((ΔP₁ₜ/100000),digits=3) round((ΔP₂ₜ/100000),digits=3) round(PPₜ,digits=2) round(Qae,digits=2) round(Qai,digits=2) round(Qₘe,digits=2) round(Volume,digits=6) round(Weight,digits=3) round(HXVolume, digits=6)]
    CSV.write(output, DataFrame(OutParameters);append=true)
    close(output)
    
    
    @goto finishit
    
    @label endit
    output = open("HXTestData.csv", "a")
    CSV.File(output; header=false)
    CSV.File(output; dateformat="mm/dd/yyyy")
    Time = Dates.now()
    Time = Dates.format(Time, "e, dd u yyyy HH:MM:SS")
    OutParameters = [Int(counter) Time "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" round(NDₛ,digits=3) round(Dₒₛ,digits=3) round(Dᵢₚ,digits=6) round(Dₜ,digits=6) round(ExD,digits=4) round(Nₜₜ,digits=2) round(NbT,digits=2) round(tₖ,digits=6) round(Nₜⱼ,digits=2) round(Nₜᵢ,digits=2) round(Pⱼ,digits=6) round(Pᵢ,digits=6) round(Lₖ,digits=3) round(Df,digits=6) round(δfb,digits=8) round(δft,digits=8) round(Pf,digits=8) "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints" "Violates constraints"]
    CSV.write(output, DataFrame(OutParameters);append=true)
    close(output)
    
    @goto finishit
    
    @label pressexcess
    print("Pressure drop exceeds the input value!")
    @goto finishit
    
    @label finishit   
    
    return T₁total,T₂total,Tₛtotal,Tₛₒtotal,Tₛᵢtotal,Nₖ,Nⱼ,Nₜᵢ,Nₜⱼ,NbT,Pⱼ,Pᵢ
    
end

### Main function calling the HX function

In [ ]:
function main()
    
    Dₜ = 0.002  # Tube outer diameter [m]
    PᵢoDₜ = 1.50  # Pitch/diameter of tube in the x direction (columns) [-]
    PⱼoDₜ = 2.90  # Pitch/diameter of tube in the y direction (rows) [-]
    NbL = 5 # Number of baffles in the left side of the HX [-]
    NbR = 5 # Number of baffles in the right side of the HX [-]
    Nₜₜ = 100 # Total number of tubes on one side of the HX [-]
    Lₖ = 0.5  # Length of heat exchanger [m]
    
    tₛ = 0.001 # Thickness of longitudinal baffle on one side [m]
    Bᵪ = 0.5 # Percentage of shell height occupied by tubes [-]
    tₖ = 0.0005  # Baffle thickness [m]
    BₐL = 1.0 # Baffle spacing factor in the left side of the HX (0-1) [-]
    BₐR = 1.0 # Baffle spacing factor in the right side of the HX (0-1) [-]
    NF = 1 # Number of folds [-]
    Np = 1 # Number of tube passes [-]
    
    ṁₕ = 0.1  # Hot side mass flow (External) [kg/s]
    ṁₗ = 0.1  # Cold side mass flow (Internal)[kg/s]
    Ratio = 1.00  # (ṁₕ*Cₚ₍ₕ₎/(ṁₗ*Cₚ₍ₗ₎)
    
    T₁₍ᵢₙ₎ = 800.00  # Hot inlet temperature (External) [°C]
    T₂₍ᵢₙ₎ = 300.00  # Cold inlet temperature (Internal) [°C]
    P₁₍ᵢₙ₎ = 8e6  # External flow inlet pressure [Pa]
    P₂₍ᵢₙ₎ = 25e6  # Internal flow inlet pressure [Pa]
    
    FinType = 0 # 0 for bare tubes, 1 for disc fins, 2 for pin-fins
    
    NAF = Int(10) # Total number of cylindrical pin-fins in the annular direction on one tube; 0 for disc fins and bare tubes
    
    DfoDₜ = 1.2 # Fin diameter to tube OD ratio [m] (Varies between 1 to min(Pd,Pl,Pt)); 1 for bare tubes
    Df = DfoDₜ * Dₜ # Fin diameter [m]
    δfboDt = 0.05 # Fin base thickness to fin diameter ratio [-] (varies between 0 to 0.1); 0 for bare tubes
    δfb = δfboDt * Dₜ # Fin base thickness [m]
    δft = δfb # Fin tip thickness (same as fin base thickness for disc fins and cylindrical pin-fins) [m]
    Pfoδfb = 4.0 # Fin pitch to fin base thickness ratio [-] (varies between 2 to 10)
    Pf = Pfoδfb * δfb # Fin pitch [m]
    
    CtoDt = 0.0 # Cruciform thickness to tube OD ratio [-] (between 0-0.2); 0 for tubes w/o internal augmentation
    Ct = CtoDt*Dₜ # Cruciform thickness [m]
    Ncruc = 4 # Number of "longitudinal internal fins" (cruciform)
    
    if FinType == 0
        DfoDₜ = 1.0 # Fin diameter to tube OD ratio [m] (Varies between 1 to min(Pd,Pl,Pt)); 1 for bare tubes
        Df = DfoDₜ * Dₜ # Fin diameter [m]
        δfboDt = 0.0 # Fin base thickness to fin diameter ratio [-] (varies between 0 to 0.1); 0 for bare tubes
        δfb = δfboDt * Dₜ # Fin base thickness [m]
        δft = δfb # Fin tip thickness (same as fin base thickness for disc fins and cylindrical pin-fins) [m]
        Pfoδfb = 4.0 # Fin pitch to fin base thickness ratio [-] (varies between 2 to 10)
        Pf = Pfoδfb * δfb # Fin pitch [m]
    end
    
    CtoDt = 0.0 # Cruciform thickness to tube OD ratio [-] (between 0-0.2); 0 for tubes w/o internal augmentation
    Ct = CtoDt*Dₜ # Cruciform thickness [m]
    Ncruc = 4 # Number of "longitudinal internal fins" (cruciform)
    
    ExD = 0.01 # Extra space taken by the side longitudinal baffles [m]
    
    ttoDt = 0.2 # Ratio of tube thickness to tube OD [-]
    tsoDs = 0.1 # Ratio of shell thickness to shell ID [-]
    Hₜ = 0.01 # Header thickness [m]
    δsb = 0.0 # Diametral clearance between shell and baffle [m]
    δtb = 0.0508*0.001 # Diametral clearance between tube and baffle hole [m]
    tsb = 0.00254 # Thickness of side longitudinal baffle [m]
    
    Bypass = 1 # 0 to consider bypass effects; 1 to neglect bypass effects
    
    Nss = 0 # Number of sealing strips [-]
    
    if Ct<0.0000001
        Ct = 0
        Ncruc = 0
        Ch = 0
    end
    
    if δfb<0.0000001 || DfoDₜ<1.0001
        Pf = 0
        δfb = 0
        Df = Dₜ
    end
    
    if (FinType == 0 || FinType == 1 || FinType == 2)
    else
        print("Please re-run with a valid external fin type")
    end
    
    counter = 1
    
    CaseRun = 0
    
    IJulia.clear_output(true)
    
    Header=["HX No." "Date" "Inlet Te [°C]" "Inlet Ti [°C]" "Outlet Te [°C]" "Outlet Ti [°C]" "ϵe [-]" "ϵi [-]" "Shell Di [m]" "Shell Do [m]" "Tube Di [m]" "Tube Do [m]" "ExD [m]" "Number of tubes [-]" "Number of baffles [-]" "Baffle thickness [m]" "Number of rows [-]" "Number of columns [-]" "Longitudinal pitch [m]" "Transverse pitch [m]" "Length of HX [m]" "Fin diameter [m]" "Fin base thickness [m]" "Fin tip thickness [m]" "Fin pitch [m]" "Cruciform thickness [m]" "Re internal [-]" "Re external [-]" "External frontal velocity [m/s]" "Internal flow velocity [m/s]" "HTC internal [W/m²K]" "HTC external [W/m²K]" "Window ΔPe [bar]" "Tube bundle ΔPe" "Core ΔP/V [-]" "ΔPe [bar]" "ΔPi [bar]" "Pumping power [W]" "Total power (ext) [W]" "Total power (int) [W]" "Maximum available power [W]" "Solid Volume [m³]" "HX Weight [kg]" "HX Volume [m³]"]
    output = open("HXTestData.csv", "a")
    CSV.File(output; header=false)
    CSV.write(output, DataFrame(Header);append=true)
    close(output)
    
    @time Outputs = HX(Dₜ, PᵢoDₜ, PⱼoDₜ, NbL, NbR, Nₜₜ, Lₖ, tₛ, Bᵪ, tₖ, BₐL, BₐR, NF, Np, ṁₕ, T₂₍ᵢₙ₎, T₁₍ᵢₙ₎, Ratio, CaseRun, counter, Df, δfb, δft, Pf, Ct, Ncruc, ExD,P₁₍ᵢₙ₎,P₂₍ᵢₙ₎,ṁₗ, ttoDt, tsoDs, Hₜ, δsb, δtb, Nss, tsb, Bypass, FinType, NAF)
    
end 

### Run the code

In [ ]:
Outputs = main();

### Plot temperature contours

In [ ]:
T1 = Outputs[1];
T2 = Outputs[2];
Ts = Outputs[3];
Tso = Outputs[4];
Tsi = Outputs[5];
zgrid = Outputs[6];
ygrid = Outputs[7];
Ncolumns = Outputs[8];
Nrows = Outputs[9];
Nbaffles = Outputs[10];
Pl = Outputs[11];
Pt = Outputs[12];

In [ ]:
T1updated = ones(ygrid,(zgrid*(Nbaffles+2)),Ncolumns)

for i in 1:ygrid
    for j in 1:(zgrid*(Nbaffles+2)) #(Nk*Nbaffles)
        for k in 1:Ncolumns
            
            Diff = j
            while Diff>(2*ygrid)
                Diff = Diff-(2*ygrid)
            end
            
            if Diff<(zgrid+1)
                UCno = Int((j-Diff)/ygrid)+1
            else
                UCno = Int((j-Diff+ygrid)/ygrid)+1
            end
            
            if iseven(UCno)
                T1updated[i,j,k] = T1[(ygrid-i+1),((UCno*ygrid)-Diff+ygrid+1),k]
            else
                T1updated[i,j,k] = T1[i,j,k]
            end
        end
    end
end

T2yavg = ones(1,(zgrid*(Nbaffles+2)),Ncolumns)

for i in 1:Ncolumns
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:ygrid
            temp = temp + T2[j,k,i]
        end
        T2yavg[1,k,i] = temp/ygrid
    end
end

Tsyavg = ones(1,(zgrid*(Nbaffles+2)),Ncolumns)

for i in 1:Ncolumns
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:ygrid
            temp = temp + Ts[j,k,i]
        end
        Tsyavg[1,k,i] = temp/ygrid
    end
end

Tsoyavg = ones(1,(zgrid*(Nbaffles+2)),Ncolumns)

for i in 1:Ncolumns
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:ygrid
            temp = temp + Tso[j,k,i]
        end
        Tsoyavg[1,k,i] = temp/ygrid
    end
end

Tsiyavg = ones(1,(zgrid*(Nbaffles+2)),Ncolumns)

for i in 1:Ncolumns
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:ygrid
            temp = temp + Tsi[j,k,i]
        end
        Tsiyavg[1,k,i] = temp/ygrid
    end
end

ygridnew = Int(ygrid*20)

totTyz = ones(ygridnew,(zgrid*(Nbaffles+2)),Ncolumns)

for i in 1:Ncolumns
    for k in 1:(zgrid*(Nbaffles+2))
        for jt in 1:ygridnew
            
            if jt>(Int((ygridnew/4)+6))
                j = Int(round((jt/10)-(ygrid/2),digits=0))
            else
                j = 1
            end
            
            if jt>(Int(ygridnew/4)) && jt<(Int((3*ygridnew/4)+1))
                if j<(Int((ygrid/2)-14)) || j>(Int((ygrid/2)+15))
                    totTyz[jt,k,i] = T1updated[j,k,i]
                elseif j == (Int((ygrid/2)-10)) || j == (Int((ygrid/2)+11))
                    totTyz[jt,k,i] = Tsiyavg[1,k,i]
                elseif j > (Int((ygrid/2)-10)) && j < (Int((ygrid/2)+11))
                    totTyz[jt,k,i] = T2yavg[1,k,i]
                else
                    totTyz[jt,k,i] = Tsoyavg[1,k,i]
                end
            end
            
            Diff = k
            while Diff>(2*ygrid)
                Diff = Diff-(2*ygrid)
            end
            
            if Diff<(zgrid+1)
                UCno = Int((k-Diff)/ygrid)+1
            else
                UCno = Int((k-Diff+ygrid)/ygrid)+1
            end
            
            if isodd(UCno)
                if jt<(Int((ygridnew/4)+1))
                    totTyz[jt,k,i] = (T1[1,Int(((UCno*ygrid)-(ygrid/2))),i])
                elseif jt>(Int((3*ygridnew/4)))
                    totTyz[jt,k,i] = (T1[ygrid,Int(((UCno*ygrid)-(ygrid/2))),i])
                end
            else
                if jt<(Int((ygridnew/4)+1))
                    totTyz[jt,k,i] = (T1[ygrid,Int(((UCno*ygrid)-(ygrid/2))),i])
                elseif jt>(Int((3*ygridnew/4)))
                    totTyz[jt,k,i] = (T1[1,Int(((UCno*ygrid)-(ygrid/2))),i])
                end
            end
            
        end
    end
end

In [ ]:
ys = [string("Ny ",i) for i in 1:ygridnew]
zs = [string("Nz ",i) for i in 1:(zgrid*(Nbaffles+2))]
plot3 = heatmap(totTyz[:,:,1], yflip=false, c=cgrad([:blue4,:blue1,:cornflowerblue,:aqua,:aquamarine3,:lime,:yellow,:orange,:red,:maroon]))

pointsz= ones(2,(Nbaffles+1))
pointsy = ones(2,(Nbaffles+1))

for i in 1:(Nbaffles+1)
    pointsz[1,i] = i*ygrid
    pointsz[2,i] = i*ygrid
    
    if iseven(i)
        pointsy[1,i] = ygridnew
        pointsy[2,i] = Int(ygridnew/4)
    else
        pointsy[1,i] = 0
        pointsy[2,i] = Int(3*ygridnew/4)
    end
end

for i in 1:(Nbaffles+1)
    plot3 = plot!(pointsz[:,i], pointsy[:,i], c=([:black]), line=4)
end

points1z = [zgrid 0 0 0 (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2));(zgrid*(Nbaffles+1)) (zgrid*(Nbaffles+2)) 0 0 (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2))]
points1y = [0 ygridnew 0 (Int((ygridnew/2)+105)) 0 (Int((ygridnew/2)+105));0 ygridnew (Int((ygridnew/2)-95)) ygridnew (Int((ygridnew/2)-95)) ygridnew]

for i in 1:6
    plot3 = plot!(points1z[:,i], points1y[:,i], c=([:black]), line=4)
end

points2z = [0 0 0 0;(zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2))]
points2y = [(Int((ygridnew/2)-145)) (Int((ygridnew/2)-95)) (Int((ygridnew/2)+105)) (Int((ygridnew/2)+155));(Int((ygridnew/2)-145)) (Int((ygridnew/2)-95)) (Int((ygridnew/2)+105)) (Int((ygridnew/2)+155))]

for i in 1:4
    plot3 = plot!(points2z[:,i], points2y[:,i], c=([:black]), line=2, legend=false)
end

plot!(grid=false, axis=nothing, border=:none, size = (950, 250))

@show plot3

In [ ]:
xgrid = Int(Ncolumns)

T2xavg = ones(ygrid,(zgrid*(Nbaffles+2)),1)

for i in 1:ygrid
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:Ncolumns
            temp = temp + T2[i,k,j]
        end
        T2xavg[i,k,1] = temp/Ncolumns
    end
end

Tsxavg = ones(ygrid,(zgrid*(Nbaffles+2)),1)

for i in 1:ygrid
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:Ncolumns
            temp = temp + Ts[i,k,j]
        end
        Tsxavg[i,k,1] = temp/Ncolumns
    end
end

Tsoxavg = ones(ygrid,(zgrid*(Nbaffles+2)),1)

for i in 1:ygrid
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:Ncolumns
            temp = temp + Tso[i,k,j]
        end
        Tsoxavg[i,k,1] = temp/Ncolumns
    end
end

Tsixavg = ones(ygrid,(zgrid*(Nbaffles+2)),1)

for i in 1:ygrid
    for k in 1:(zgrid*(Nbaffles+2))
        temp = 0.0
        for j in 1:Ncolumns
            temp = temp + Tsi[i,k,j]
        end
        Tsixavg[i,k,1] = temp/Ncolumns
    end
end

xgridnew = Int(xgrid*20)
totTxz = ones(ygrid,(zgrid*(Nbaffles+2)),xgridnew)

for j in 1:ygrid
    for k in 1:(zgrid*(Nbaffles+2))
        for it in 1:xgridnew
        
            if it>11
                i = Int(round((it/20),digits=0))
            else
                i = 1
            end
            
            if i<(Int(round((xgrid/2)-(Int(round(((xgrid/10)+2),digits=0))),digits=0))) || i>(Int(round((xgrid/2)+(Int(round(((xgrid/10)+2),digits=0))),digits=0)))
                totTxz[j,k,it] = T1updated[j,k,i]
            elseif i==(Int(round((xgrid/2)-(Int(round(((xgrid/10)+2),digits=0))),digits=0))) || i==(Int(round((xgrid/2)+(Int(round(((xgrid/10)+2),digits=0))),digits=0)))
                totTxz[j,k,it] = Tsoxavg[j,k,1]
            elseif i==(Int(round((xgrid/2)-(Int(round(((xgrid/10)+1),digits=0))),digits=0))) || i==(Int(round((xgrid/2)+(Int(round(((xgrid/10)+1),digits=0))),digits=0)))
                totTxz[j,k,it] = Tsixavg[j,k,1]
            else
                totTxz[j,k,it] = T2xavg[j,k,1]
            end
            
            
        end
    end
end

In [ ]:
### xgridnew = Int(xgrid*20)
xs = [string("Ny ",i) for i in 1:xgridnew]
zs = [string("Nz ",i) for i in 1:(zgrid*(Nbaffles+2))]
plot2 = heatmap(zs, xs, totTxz[8,:,:], transpose=true, yflip=false, c=cgrad([:blue4,:blue1,:cornflowerblue,:aqua,:aquamarine3,:lime,:yellow,:orange,:red,:maroon]))

pointsz = ones(2,(Nbaffles+1))
pointsx = ones(2,(Nbaffles+1))

for i in 1:(Nbaffles+1)
    pointsz[1,i] = i*ygrid
    pointsz[2,i] = i*ygrid
    
    if iseven(i)
        pointsx[1,i] = xgridnew
        pointsx[2,i] = 0
    else
        pointsx[1,i] = 0
        pointsx[2,i] = xgridnew
    end
end

for i in 1:(Nbaffles+1)
    plot2 = plot!(pointsz[:,i], pointsx[:,i], c=([:black]), line=4)
end

points1z = [0 0 0 0 (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2));(zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2)) 0 0 (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2))]
points1y = [0 xgridnew 0 ((Int(round((xgrid/2)+(Int(round(((xgrid/10)+1),digits=0))),digits=0)))-0.5)*20 0 ((Int(round((xgrid/2)+(Int(round(((xgrid/10)+1),digits=0))),digits=0)))-0.5)*20;0 xgridnew ((Int(round((xgrid/2)-(Int(round(((xgrid/10)+1),digits=0))),digits=0)))+0.5)*20 xgridnew ((Int(round((xgrid/2)-(Int(round(((xgrid/10)+1),digits=0))),digits=0)))+0.5)*20 xgridnew]

for i in 1:6
    plot2 = plot!(points1z[:,i], points1y[:,i], c=([:black]), line=4)
end

points2x = [0 0 0 0;(zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2)) (zgrid*(Nbaffles+2))]
points2z = [((Int(round((xgrid/2)-(Int(round(((xgrid/10)+2),digits=0))),digits=0)))-0.5)*20 ((Int(round((xgrid/2)-(Int(round(((xgrid/10)+1),digits=0))),digits=0)))+0.5)*20 ((Int(round((xgrid/2)+(Int(round(((xgrid/10)+1),digits=0))),digits=0)))-0.5)*20 ((Int(round((xgrid/2)+(Int(round(((xgrid/10)+2),digits=0))),digits=0)))+0.5)*20;((Int(round((xgrid/2)-(Int(round(((xgrid/10)+2),digits=0))),digits=0)))-0.5)*20 ((Int(round((xgrid/2)-(Int(round(((xgrid/10)+1),digits=0))),digits=0)))+0.5)*20 ((Int(round((xgrid/2)+(Int(round(((xgrid/10)+1),digits=0))),digits=0)))-0.5)*20 ((Int(round((xgrid/2)+(Int(round(((xgrid/10)+2),digits=0))),digits=0)))+0.5)*20]

for i in 1:4
    plot2 = plot!(points2x[:,i], points2z[:,i], c=([:black]), line=2, legend=false)
end

plot!(grid=false, axis=nothing, border=:none, size = (950, 250))

@show plot2